# Calculate Advanced Fantasy Football Metrics

In [81]:
# import needed packages
import numpy as np
import pandas as pd

# set pandas display options
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)
pd.set_option('display.width',1000)

#### Read in Roster Data w/ Fantasy Football Statistics

In [82]:
# read in rosters_df_w_scoring.csv
df = pd.read_csv("rosters_df_w_scoring.csv")

# update year and week columns' data type to category 
df['year'] = df['year'].astype('category')
df['week'] = df['week'].astype('category')

#### Pre-Process Player Names

In order to join the roster dataframe and the play-by-play dataframe, we need to create a new column in our roster dataframe called short_name.

short_name = the player's first initial followed by a period and the player's last name excluding any suffixes such as II or Jr.  For example, P.Mahomes

For the vast majority of players this format is correct, however, there are a handful of players that deviate from this pattern.  This is typically because there are two players on the same team that would have the same short_name if they both followed this pattern.  For example, the Chiefs have two running backs named Darrell Williams and Damien Williams.  

To differentiate between the two players, the play-by-play dataframe listed them as Dar.Williams or Dam.Williams depending on the year or even week.

The code below shows the transformations made to players' names to make sure their play-by-play data can be joined to their fantasy football data.

In [83]:
# extract player's first initial of their first name
df['first_initial'] = df['player'].astype(str).str[0]

# extract player's last name but taking the text between the first 2 spaces in the full player name
df['last_name'] = df['player'].str.extract(r'(?<=\s)(.*?)(?=$|\s)')

# combine first_initial and last_name
df['short_name'] = df[['first_initial', 'last_name']].agg('.'.join, axis=1)

# update short_name depending on player, team, year, and sometimes week
df.loc[(df['player'] == 'Damien Williams')  & (df['pro_team_abv'] == 'KC')   
                                            & (df['year'] == 2018), 'short_name'] = 'Dam.'
df.loc[(df['player'] == 'Darrel Williams')  & (df['pro_team_abv'] == 'KC')   
                                            & (df['year'] == 2018), 'short_name'] = 'Dar.Williams'
df.loc[(df['player'] == 'Damien Williams')  & (df['pro_team_abv'] == 'KC')   
                                            & (df['year'] == 2019), 'short_name'] = 'Dam.Williams'
df.loc[(df['player'] == 'Trey Edmunds')     & (df['pro_team_abv'] == 'PIT')  
                                            & (df['year'] == 2019) & (df['week'] == 12), 'short_name'] = 'Tr.Edmunds'
df.loc[(df['player'] == 'David Johnson')    & (df['pro_team_abv'] == 'HOU')  
                                            & (df['year'] == 2020) & (df['week'] == 2), 'short_name'] = 'Da.Johnson'
df.loc[(df['player'] == 'David Johnson')    & (df['pro_team_abv'] == 'HOU')  
                                            & (df['year'] == 2020) & (df['week'] == 3), 'short_name'] = 'Da.Johnson'
df.loc[(df['player'] == 'David Johnson')    & (df['pro_team_abv'] == 'HOU')  
                                            & (df['year'] == 2020) & (df['week'] == 13), 'short_name'] = 'Da.Johnson'
df.loc[(df['player'] == 'Duke Johnson')     & (df['pro_team_abv'] == 'HOU')  
                                            & (df['year'] == 2020) & (df['week'] == 13), 'short_name'] = 'Du.Johnson'
df.loc[(df['player'] == 'Duke Johnson')     & (df['pro_team_abv'] == 'HOU')  
                                            & (df['year'] == 2020) & (df['week'] == 14), 'short_name'] = 'Du.Johnson'
df.loc[(df['player'] == 'C.J. Anderson')    & (df['pro_team_abv'] == 'LV')   
                                            & (df['year'] == 2018), 'pro_team'] = 'Los Angeles Rams'
df.loc[(df['player'] == 'C.J. Anderson')    & (df['pro_team_abv'] == 'LV')   
                                            & (df['year'] == 2018), 'pro_team_abv'] = 'LA'
df.loc[(df['player'] == 'Demaryius Thomas') & ((df['pro_team_abv'] == 'HOU') | (df['pro_team_abv'] == 'DEN')) 
                                            & (df['year'] == 2018), 'short_name'] = 'De.Thomas'
df.loc[(df['player'] == 'Demaryius Thomas') & (df['pro_team_abv'] == 'DEN')  
                                            & (df['year'] == 2018) & (df['week'] == 9), 'pro_team'] = 'Houston Texans'
df.loc[(df['player'] == 'Demaryius Thomas') & (df['pro_team_abv'] == 'DEN')  
                                            & (df['year'] == 2018) & (df['week'] == 9), 'pro_team_abv'] = 'HOU'
df.loc[(df['player'] == 'Tyrell Williams')  & (df['pro_team_abv'] == 'LAC')  
                                            & (df['year'] == 2018), 'short_name'] = 'Ty.Williams'
df.loc[(df['player'] == 'Robby Anderson')   & (df['pro_team_abv'] == 'NYJ')  
                                            & (df['year'] == 2019), 'short_name'] = 'Ro.Anderson'
df.loc[(df['player'] == 'DK Metcalf')       & (df['pro_team_abv'] == 'SEA')  
                                            & (df['year'] == 2019), 'short_name'] = 'DK.Metcalf'
df.loc[(df['player'] == 'Delanie Walker')   & (df['pro_team_abv'] == 'TEN')  
                                            & (df['year'] == 2019), 'short_name'] = 'De.Walker'
df.loc[(df['player'] == 'Diontae Johnson')  & (df['pro_team_abv'] == 'PIT')  & (df['year'] == 2020) 
                                            & ((df['week'] == 14) | (df['week'] == 15) | (df['week'] == 16) 
                                            | (df['week'] == 17)), 'short_name'] = 'Di.Johnson'
df.loc[(df['player'] == 'Tyron Johnson')    & (df['pro_team_abv'] == 'LAC')  & (df['year'] == 2020) & ((df['week'] == 12) 
                                            | (df['week'] == 13) | (df['week'] == 14) | (df['week'] == 15) 
                                            | (df['week'] == 16) | (df['week'] == 17)), 'short_name'] = 'Ty.Johnson'
df.loc[(df['player'] == 'Josh Allen')       & (df['pro_team_abv'] == 'BUF')  & (df['year'] == 2019) & ((df['week'] == 9) 
                                            | (df['week'] == 10) | (df['week'] == 11)), 'short_name'] = 'Jos.Allen'

df.head(3)

,year,week,owner_team,owner,player,pro_team,pro_team_abv,current_inj_status,lineup_slot_name,position_name,proj_points,actual_points,slot_id,pass_comp_ff,pass_incomp_ff,pass_td_ff,pass_5_yrd_ff,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_2pt_con_ff,pass_int_ff,rush_td_ff,rush_2pt_con_ff,rush_5_yrd_ff,rush_50_yrd_td_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rec_td_ff,rec_2pt_con_ff_ff,rec_50_yrd_td_ff,rec_5_yrd_ff,receptions_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,fum_lost_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_0_39_ff,fg_miss_0_39_ff,pat_made_ff,pat_miss_ff,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_int_ff,def_st_fum_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_sack_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_46+_pts_alw_ff,def_st_0_99_yrd_alw_ff,def_st_100_199_yrd_alw_ff,def_st_200_299_yrd_alw_ff,def_st_350_399_yrd_alw_ff,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,fg_made_50_59_ff,pass_att,pass_comp,pass_incomp,pass_yrd,pass_td,pass_5_yrd,unk6,unk7,unk8,unk9,unk10,unk11,unk12,unk13,unk14,unk15,pass_50_yrd_td,pass_yrd_300_399,pass_yrd_400+,pass_2pt_con,pass_int,unk21,pass_yrd_dupe,rush_att,rush_yrd,rush_td,rush_2pt_con,rush_5_yrd,unk28,unk29,unk30,unk31,unk32,unk33,unk34,unk35,rush_50_yrd_td,rush_yrd_100_199,rush_yrd_200+,unk39,unk40,receptions_dupe,rec_yrd,rec_td,rec_2pt_con,unk45,rec_50_yrd_td,rec_5_yrd,unk48,unk49,unk50,unk51,unk52,receptions,unk54,unk55,rec_yrd_100_199,rec_yrd_200+,rec_tar,yac,yrd_per_rec,rec_yrd_dupe,unk62,unk64,unk65,unk66,unk67,unk68,unk69,unk70,unk71,fum_lost,unk73,fg_made_50+,unk75,unk76,fg_made_40_49,unk78,fg_miss_40_49,fg_made_0_39,unk81,fg_miss_0_39,fg_con,fg_att,fg_miss_tot,pat_con,pat_att,pat_miss_tot,def_st_0_pts_alw,def_st_1_6_pts_alw,def_st_7_13_pts_alw,def_st_14_17_pts_alw,def_st_blk_td,unk94,def_st_int,def_st_fum,def_st_blk_kick,def_st_safety,def_st_sack,unk100,def_st_kick_ret_td,def_st_punt_ret_td,def_st_int_td,def_st_fum_ret_td,unk105,unk106,unk107,unk108,unk109,unk110,unk111,unk112,unk113,unk114,unk115,unk116,unk117,unk118,unk119,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_st_46+_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk155,unk156,unk158,unk175,unk176,unk177,unk178,unk179,unk180,unk181,unk182,unk183,unk184,unk185,unk186,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk196,unk197,fg_made_50_59,unk199,unk200,unk202,unk203,unk210,first_initial,last_name,short_name
0,2018,1,Happy Rock Homewreckers,Blainer,David Johnson,Arizona Cardinals,ARI,ACTIVE,RB,RB,21.371488,15.8,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.2,0.0,0.0,0.0,0.0,0.0,0.0,3.6,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,37.0,1.0,0.0,7.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.111111,37.0,5.0,30.0,0.0,0.0,0.0,0.0,6.0,3.0,1.0,1.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,9.0,0.0,6.000000,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,D,Johnson,D.Johnson
1,2018,1,Happy Rock Homewreckers,Blainer,Melvin Gordon,Los Angeles Chargers,LAC,ACTIVE,RB,RB,16.624250,27.8,2,0.0,0.0,0.0,0.0,0.0,0.0

#### Read in Play-by-Play Data

Since the ESPN Fantasy Football only goes back to the 2018 season on its API v3, we'll only read in play-by-play data from the 2018 season through the 2020 season.

In [84]:
# load play by play data by reading directly from source
#play_by_play_df = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/play_by_play_2018.csv.gz?raw=True', compression='gzip', low_memory=False)
#play_by_play_df.to_csv("play_by_play_2018.csv")
#play_by_play_df.head()

In [85]:
# load play by play data from csv
play_by_play_2018 = pd.read_csv("play_by_play_2018.csv", low_memory=False)
play_by_play_2018['year'] = 2018
play_by_play_2019 = pd.read_csv("play_by_play_2019.csv", low_memory=False)
play_by_play_2019['year'] = 2019
play_by_play_2020 = pd.read_csv("play_by_play_2020.csv", low_memory=False)
play_by_play_2020['year'] = 2020
play_by_play_df = play_by_play_2018.append(play_by_play_2019, ignore_index=True)\
                                   .append(play_by_play_2020, ignore_index=True)
play_by_play_df.head(2)

,Unnamed: 0,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,

In [86]:
play_by_play_df.tail(2)

,Unnamed: 0,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,

In [87]:
list(play_by_play_df.columns)

['Unnamed: 0',
 'play_id',
 'game_id',
 'old_game_id',
 'home_team',
 'away_team',
 'season_type',
 'week',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'td_player_name',
 'td_player_id',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_diff

# Create Dataframe for Running Backs, Wide Receivers, and Tight Ends

##### Advanced Rushing Metrics:
* **% of Team Rushing Attempts (rush_share)** = num of individual rushing attempts / num of team rushing attempts
* **Yards per Rushing Attempt** = total individual rushing yards / num of individual rushing attempts

##### Advanced Receiving Metrics:
* **% of Team Targets (target_share)** = num of individual targets / num of team targets
* **Weighted Opportunity Rating (wopr)** = (1.5 * target share) + (0.7 * share of team air yards)
* **Average Depth of Target (adot)** = total air yards / num of individual targets
* **Receiver Air Conversion Ratio (racr)** = total receiving yards / total air yards
* **Completed Air Yards (compl_rec_air_yrds)** = total receiving yards - yards after catch
* **Yards per Catch (yrd_per_catch)** = total receiving yards / total catches


##### Advanced Metrics for RB, WR, and TE:
* **% of Total Offense (total_off_share)** = (rush attempts + targets) / (team rush attempts + team targets)

Sources:
* https://www.4for4.com/2018/preseason/air-yards-explained
* https://noextrapoints.com/air-yards-and-other-metrics
* https://www.fantasyfootballdatapros.com/blog/intermediate/12

### Create Dataframe of Rushing Plays (for QBs, RBs, WRs, & TEs) and Compute Advanced Rushing Metrics

In [88]:
# filter for rush plays only and select relevant columns
rb_df = play_by_play_df.loc[play_by_play_df['rush_attempt'] == 1, ['rusher_player_id'
                                                                  ,'rusher_player_name'
                                                                  ,'posteam'
                                                                  ,'year'
                                                                  ,'week'
                                                                  ,'rush_attempt'
                                                                  ,'rushing_yards'
                                                                  ,'yards_gained'
                                                                  ,'touchdown'
                                                                  ]
                           ]

# remove rows with no rusher_player_id
rb_df = rb_df.loc[rb_df['rusher_player_id'].notnull()]

# group by player, team, week, year and sum the results for a weekly stat line for each player
rb_df = rb_df.groupby(['rusher_player_id', 'rusher_player_name', 'posteam', 'year', 'week'], as_index=False).sum()

# rename rush_attempt column
rb_df = rb_df.rename({'rush_attempt': 'rush_attempts'}, axis=1)

# calculate team rushing attempts by year and week
team_rush = rb_df.groupby(['posteam', 'year', 'week'], as_index=False)[['rush_attempts']].sum()

# merge the team_rush dataframe with the rb_df dataframe
rb_df = rb_df.merge(team_rush, on=['posteam', 'year', 'week'], how='left', suffixes=('_ind', '_team'))

# calculate % of team rushing attempts
rb_df['rush_share'] = rb_df['rush_attempts_ind'] / rb_df['rush_attempts_team']

# calculate yards per rushing attempt
rb_df['yrd_per_rush'] = rb_df['rushing_yards'] / rb_df['rush_attempts_ind']

# remove any text after the players' last names such as Jr or II.  needed to join logic
rb_df['rusher_player_name'] = rb_df['rusher_player_name'].str.extract(r'(.*?)(?=$|\s)')

rb_df.head(3)

,rusher_player_id,rusher_player_name,posteam,year,week,rush_attempts_ind,rushing_yards,yards_gained,touchdown,rush_attempts_team,rush_share,yrd_per_rush
0,00-0019596,T.Brady,NE,2018,1,1.0,2.0,2.0,0.0,31.0,0.032258,2.000000
1,00-0019596,T.Brady,NE,2018,2,3.0,10.0,10.0,0.0,24.0,0.125000,3.333333
2,00-0019596,T.Brady,NE,2018,3,1.0,2.0,2.0,0.0,19.0,0.052632,2.000000


### Create Dataframe of Passing Plays (for RBs, WRs & TEs) and Compute Advanced Receiving Metrics

In [89]:
# filter for passing plays only and select relevant columns
rec_df = play_by_play_df.loc[play_by_play_df['pass_attempt'] == 1, ['receiver_player_id'
                                                                   ,'receiver_player_name'
                                                                   ,'posteam'
                                                                   ,'year'
                                                                   ,'week'
                                                                   ,'pass_attempt'
                                                                   ,'complete_pass'
                                                                   ,'yards_after_catch'
                                                                   ,'yards_gained'
                                                                   ,'touchdown'
                                                                   ,'air_yards'
                                                                   ]
                            ]

# remove rows with no receiver_player_id
rec_df = rec_df.loc[rec_df['receiver_player_id'].notnull()]

# group by player, team, week, year and sum the results for a weekly stat line for each player
rec_df = rec_df.groupby(['receiver_player_id', 'receiver_player_name', 'posteam', 'year', 'week'], as_index=False).sum()

# rename pass_attempt column
rec_df = rec_df.rename({'pass_attempt': 'target'}, axis=1)

# calculate team targets and air yards by year and week
team_targets = rec_df.groupby(['posteam', 'year', 'week'], as_index=False)[['target']].sum()
team_ay = rec_df.groupby(['posteam', 'year', 'week'], as_index=False)[['air_yards']].sum()

# merge team_targets and team_ay with rec_df
rec_df = rec_df.merge(team_ay, on=['posteam', 'year', 'week'], how='left', suffixes=('_ind', '_team'))\
               .merge(team_targets, on=['posteam', 'year', 'week'], how='left', suffixes=('_ind', '_team'))

# calculate weighted opportunity rating
rec_df['wopr'] = ((rec_df['target_ind'] / rec_df['target_team']) * 1.5) +\
                 ((rec_df['air_yards_ind'] / rec_df['air_yards_team']) * 0.7)

# calculate average depth of target
rec_df['adot'] = rec_df['air_yards_ind'] / rec_df['target_ind']

# calculate receiver air conversion ratio
rec_df['racr'] = rec_df['yards_gained'] / rec_df['air_yards_ind']

# calculate % of team targets
rec_df['target_share'] = rec_df['target_ind'] / rec_df['target_team']

# calculate completed air yards
rec_df['compl_rec_air_yrds'] = rec_df['yards_gained'] - rec_df['yards_after_catch']

# calculate yards per catch
rec_df['yrd_per_catch'] = rec_df['yards_gained'] / rec_df['complete_pass']

# remove any text after the players' last names such as Jr or II.  needed to join logic
rec_df['receiver_player_name'] = rec_df['receiver_player_name'].str.extract(r'(.*?)(?=$|\s)')

rec_df.head(3)

,receiver_player_id,receiver_player_name,posteam,year,week,target_ind,complete_pass,yards_after_catch,yards_gained,touchdown,air_yards_ind,air_yards_team,target_team,wopr,adot,racr,target_share,compl_rec_air_yrds,yrd_per_catch
0,00-0019596,T.Brady,NE,2018,10,1.0,1.0,6.0,6.0,0.0,0.0,335.0,40.0,0.037500,0.00,inf,0.025000,0.0,6.0
1,00-0020531,D.Brees,NO,2018,5,1.0,1.0,3.0,1.0,0.0,-2.0,160.0,29.0,0.042974,-2.00,-0.500000,0.034483,-2.0,1.0
2,00-0021547,A.Gates,LAC,2018,1,4.0,2.0,5.0,18.0,0.0,27.0,483.0,52.0,0.154515,6.75,0.666667,0.076923,13.0,9.0


### Create Dataframe of RB/WR/TE Fantasy Football Statistics

In [90]:
# create column list of specific fantasy football statistics
rbwrte_ff_stats = ['rush_5_yrd_ff'
                  ,'rush_yrd_100_199_ff'
                  ,'rush_yrd_200+_ff'
                  ,'rush_td_ff'
                  ,'rush_50_yrd_td_ff'
                  ,'rush_2pt_con_ff'
                  ,'receptions_ff'
                  ,'rec_5_yrd_ff'
                  ,'rec_yrd_100_199_ff'
                  ,'rec_yrd_200+_ff'
                  ,'rec_td_ff'
                  ,'rec_50_yrd_td_ff'
                  ,'rec_2pt_con_ff_ff'
                  ,'fum_lost_ff'
                  ,'actual_points'
                  ]

# create column list of specific football statistics
rbwrte_stats = ['rush_att'
               ,'rush_yrd'
               ,'rush_td'
               ,'rush_2pt_con'
               ,'rec_tar'
               ,'receptions'
               ,'rec_yrd'
               ,'rec_td'
               ,'rec_2pt_con'
               ,'fum_lost'
               ]

# create list of player specific columns
player_columns = ['year'
                 ,'week'
                 ,'player'
                 ,'short_name'
                 ,'position_name'
                 ,'pro_team'
                 ,'pro_team_abv']

# filter by position_name is equal to RB, WR, or TE + the column lists created above
rbwrte_df = df.loc[(df['position_name'] == 'RB') | (df['position_name'] == 'WR') | (df['position_name'] == 'TE'), 
                   player_columns + rbwrte_stats + rbwrte_ff_stats
                  ]

rbwrte_df.head(3)

,year,week,player,short_name,position_name,pro_team,pro_team_abv,rush_att,rush_yrd,rush_td,rush_2pt_con,rec_tar,receptions,rec_yrd,rec_td,rec_2pt_con,fum_lost,rush_5_yrd_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rush_td_ff,rush_50_yrd_td_ff,rush_2pt_con_ff,receptions_ff,rec_5_yrd_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,rec_td_ff,rec_50_yrd_td_ff,rec_2pt_con_ff_ff,fum_lost_ff,actual_points
0,2018,1,David Johnson,D.Johnson,RB,Arizona Cardinals,ARI,9.0,37.0,1.0,0.0,9.0,5.0,30.0,0.0,0.0,0.0,4.2,0.0,0.0,6.0,0.0,0.0,2.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,15.8
1,2018,1,Melvin Gordon,M.Gordon,RB,Los Angeles Chargers,LAC,15.0,64.0,0.0,1.0,13.0,9.0,102.0,0.0,0.0,0.0,7.2,0.0,0.0,0.0,0.0,2.0,3.6,12.0,3.0,0.0,0.0,0.0,0.0,0.0,27.8
2,2018,1,Rob Gronkowski,R.Gronkowski,TE,New Engalnd Patriots,NE,0.0,0.0,0.0,0.0,8.0,7.0,123.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8,14.4,3.0,0.0,6.0,0.0,0.0,-2.0,24.2


### Merge Rushing Plays, Passing Plays, and Fantasy Football Statistics Dataframes

In [91]:
# merge rbwrte_df with rb_df
rbwrte_df = rbwrte_df.merge(rb_df, left_on = ['year', 'week', 'short_name', 'pro_team_abv'],
                                   right_on = ['year', 'week', 'rusher_player_name', 'posteam'], how = 'left')

# merge rbwrte_df with rec_df
rbwrte_df = rbwrte_df.merge(rec_df, left_on = ['year', 'week', 'short_name', 'pro_team_abv'],
                                    right_on = ['year', 'week', 'receiver_player_name', 'posteam'], how = 'left')

# calculate % of total offense 
rbwrte_df['total_off_share'] = (rbwrte_df['receptions'] + rbwrte_df['rush_att']) /\
                               (rbwrte_df['target_team'] + rbwrte_df['rush_attempts_team'])

# create list of columns to fill nulls with 0
null_to_zero = ['target_team'
               ,'rush_attempts_team'
               ,'rush_share'
               ,'yrd_per_rush'
               ,'total_off_share'
               ,'target_share'
               ,'yrd_per_catch'
               ,'yards_after_catch'
               ,'air_yards_ind'
               ,'compl_rec_air_yrds'
               ,'racr'
               ,'adot'
               ,'wopr'
               ]

# fill nulls with 0 since some players don't have targets/receptions or rushing attempts every game
rbwrte_df[null_to_zero] = rbwrte_df[null_to_zero].fillna(0)

# replace infinite values with 0
rbwrte_df = rbwrte_df.replace([-np.inf, np.inf], 0)

# create column list of advanced metrics for RB/WR/TE
rbwrte_advanced_metrics = ['yrd_per_rush'
                          ,'rush_share'
                          ,'total_off_share'
                          ,'target_share'
                          ,'yrd_per_catch'
                          ,'yards_after_catch'
                          ,'air_yards_ind'
                          ,'compl_rec_air_yrds'
                          ,'racr'
                          ,'adot'
                          ,'wopr'
                          ]

# select relevant columns
rbwrte_df = rbwrte_df[player_columns + rbwrte_stats + rbwrte_advanced_metrics + rbwrte_ff_stats]
rbwrte_df['year'] = rbwrte_df['year'].astype('category')
rbwrte_df['week'] = rbwrte_df['week'].astype('category')
rbwrte_df.head(3)

,year,week,player,short_name,position_name,pro_team,pro_team_abv,rush_att,rush_yrd,rush_td,rush_2pt_con,rec_tar,receptions,rec_yrd,rec_td,rec_2pt_con,fum_lost,yrd_per_rush,rush_share,total_off_share,target_share,yrd_per_catch,yards_after_catch,air_yards_ind,compl_rec_air_yrds,racr,adot,wopr,rush_5_yrd_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rush_td_ff,rush_50_yrd_td_ff,rush_2pt_con_ff,receptions_ff,rec_5_yrd_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,rec_td_ff,rec_50_yrd_td_ff,rec_2pt_con_ff_ff,fum_lost_ff,actual_points
0,2018,1,David Johnson,D.Johnson,RB,Arizona Cardinals,ARI,9.0,37.0,1.0,0.0,9.0,5.0,30.0,0.0,0.0,0.0,4.111111,0.600000,0.28,0.257143,6.000000,20.0,7.0,10.0,4.285714,0.777778,0.409272,4.2,0.0,0.0,6.0,0.0,0.0,2.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,15.8
1,2018,1,Melvin Gordon,M.Gordon,RB,Los Angeles Chargers,LAC,15.0,64.0,0.0,1.0,13.0,9.0,102.0,0.0,0.0,0.0,4.000000,0.695652,0.32,0.250000,11.333333,119.0,-16.0,-17.0,-6.375000,-1.230769,0.351812,7.2,0.0,0.0,0.0,0.0,2.0,3.6,12.0,3.0,0.0,0.0,0.0,0.0,0.0,27.8
2,2018,1,Rob Gronkowski,R.Gronkowski,TE,New Engalnd Patriots,NE,0.0,0.0,0.0,0.0,8.0,7.0,123.0,1.0,0.0,1.0,0.000000,0.000000,0.00,0.205128,17.571429,21.0,107.0,102.0,1.149533,13.375000,0.541755,0.0,0.0,0.0,0.0,0.0,0.0,2.8,14.4,3.0,0.0,6.0,0.0,0.0,-2.0,24.2


In [92]:
# view rbwrte_df snapshot
rbwrte_df.info()
rbwrte_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7615 entries, 0 to 7614
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   year                 7615 non-null   category
 1   week                 7615 non-null   category
 2   player               7615 non-null   object  
 3   short_name           7615 non-null   object  
 4   position_name        7615 non-null   object  
 5   pro_team             7615 non-null   object  
 6   pro_team_abv         7615 non-null   object  
 7   rush_att             7615 non-null   float64 
 8   rush_yrd             7615 non-null   float64 
 9   rush_td              7615 non-null   float64 
 10  rush_2pt_con         7615 non-null   float64 
 11  rec_tar              7615 non-null   float64 
 12  receptions           7615 non-null   float64 
 13  rec_yrd              7615 non-null   float64 
 14  rec_td               7615 non-null   float64 
 15  rec_2pt_con          

,rush_att,rush_yrd,rush_td,rush_2pt_con,rec_tar,receptions,rec_yrd,rec_td,rec_2pt_con,fum_lost,yrd_per_rush,rush_share,total_off_share,target_share,yrd_per_catch,yards_after_catch,air_yards_ind,compl_rec_air_yrds,racr,adot,wopr,rush_5_yrd_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rush_td_ff,rush_50_yrd_td_ff,rush_2pt_con_ff,receptions_ff,rec_5_yrd_ff,rec_yrd_100_199_ff,rec_yrd_200+_ff,rec_td_ff,rec_50_yrd_td_ff,rec_2pt_con_ff_ff,fum_lost_ff,actual_points
count,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000,7615.000000
mean,3.697833,16.317137,0.124491,0.004596,4.238608,2.874196,32.703349,0.208011,0.010112,0.036113,1.732434,0.141478,0.072225,0.124981,8.062095,15.433093,33.678004,17.279448,0.399235,5.106611,0.272666,1.874773,0.105187,0.006566,0.746947,0.020880,0.009192,1.743217,3.758056,0.207617,0.006566,1.248063,0.051609,0.020223,-0.072226,9.750519
std,6.407591,31.645593,0.410651,0.073237,3.838468,2.769384,37.496598,0.479212,0.100053,0.189378,3.325549,0.237886,0.122059,0.107896,7.737105,19.039911,46.869615,27.260289,5.259729,6.575779,0.260039,3.719216,0.551849,0.181083,2.463907,0.254118,0.146474,2.022352,4.443730,0.761460,0.181083,2.875273,0.396120,0.200107,0.378756,9.820568
min,0.000000,-11.000000,0.000000,0.000000,0.000000,0.000000,-8.000000,0.000000,0.000000,0.000000,-11.000000,0.000000,0.000000,0.000000,-8.000000,-8.000000,-33.000000,-34.000000,-102.000000,-9.500000,-0.042571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000,-2.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.400000
50%,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,7.500000,9.000000,10.000000,4.000000,0.472868,3.000000,0.210481,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.200000,2.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.400000
75%,5.000000,19.000000,0.000000,0.000000,7.000000,5.000000,51.000000,0.000000,0.000000,0.000000,3.200000,0.210526,0.115942,0.200000,12.333333,23.000000,59.000000,29.000000,1.107493,9.500000,0.451009,1.800000,0.000000,0.000000,0.000000,0.000000,0.000000,2.400000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.800000
max,34.000000,253.000000,6.000000,2.000000,20.000000,16.000000,269.000000,4.000000,1.000000,2.000000,49.000000,1.000000,0.666667,0.526316,75.000000,141.000000,282.000000,188.000000,95.000000,53.000000,1.310073,30.000000,3.000000,5.000000,36.000000,6.000000,4.000000,16.000000,31.800000,3.000000,5.000000,24.000000,6.000000,2.000000,0.000000,70.800000


In [93]:
# double check nulls
rbwrte_df.isnull().sum(axis = 0)

year                   0
week                   0
player                 0
short_name             0
position_name          0
pro_team               0
pro_team_abv           0
rush_att               0
rush_yrd               0
rush_td                0
rush_2pt_con           0
rec_tar                0
receptions             0
rec_yrd                0
rec_td                 0
rec_2pt_con            0
fum_lost               0
yrd_per_rush           0
rush_share             0
total_off_share        0
target_share           0
yrd_per_catch          0
yards_after_catch      0
air_yards_ind          0
compl_rec_air_yrds     0
racr                   0
adot                   0
wopr                   0
rush_5_yrd_ff          0
rush_yrd_100_199_ff    0
rush_yrd_200+_ff       0
rush_td_ff             0
rush_50_yrd_td_ff      0
rush_2pt_con_ff        0
receptions_ff          0
rec_5_yrd_ff           0
rec_yrd_100_199_ff     0
rec_yrd_200+_ff        0
rec_td_ff              0
rec_50_yrd_td_ff       0


### Save Dataframe

In [94]:
# save to csv
rbwrte_df.to_csv("rbwrte_feature_matrix.csv", index = False)

# Create Dataframe for Quarterbacks

##### Advanced Passing Metrics:
* **Completed Air Yards** = total passing yards - team yards after catch
* **Completion Percentage** = num of completed passes / num of passing attempts
* **Adjusted Net Yards per Passing Attempt** = total passing yards + (20 * num of TDs) - (45 * num of INTs) - total sack yards lost / (num of passing attempts + num of sacks taken)
* **TD to Passing Attempts Ratio** = num of TDs / num of passing attempts
* **INT to TD Ratio** = num of INTs / num of TDs
* **Passing Yards per Attempt** = total passing yards / num of passing attempts
* **Air Yards per Attempt** = total air yards / num of passing attempts
* **Total Turnovers** = INTs + fumbles lost

Source:
* https://bleacherreport.com/articles/1785998-upgrade-your-fantasy-football-approach-8-stats-to-predict-quarterback-success

### Create Dataframe of Passing Plays (for QBs)

In [95]:
# create column list for passing plays dataframe
passing_columns = ['passer_player_id'
                  ,'passer_player_name'
                  ,'posteam'
                  ,'year'
                  ,'week'
                  ,'incomplete_pass'
                  ,'complete_pass'
                  ,'pass_attempt'
                  ,'passing_yards'
                  ,'air_yards'
                  ,'yards_after_catch'
                  ,'pass_length'
                  ,'yards_gained'
                  ,'pass_location'
                  ,'touchdown'
                  ,'interception'
                  ,'fumble_lost'
                  ,'shotgun'
                  ,'no_huddle'
                  ,'qb_dropback'
                  ,'qb_kneel'
                  ,'qb_spike'
                  ,'qb_scramble'
                  ,'qb_hit'
                  ,'sack'
                  ,'ep'
                  ,'epa'
                  ,'air_epa'
                  ,'comp_air_epa'
                  ,'qb_epa'
                  ,'air_wpa'
                  ,'comp_air_wpa'
                  ,'yardline_100'
                  ]

# filter for passing plays only and select relevant columns
qb_play_by_play_df = play_by_play_df.loc[play_by_play_df['pass_attempt'] == 1, passing_columns]

# remove rows with no passer_player_id
qb_play_by_play_df = qb_play_by_play_df.loc[qb_play_by_play_df['passer_player_id'].notnull()]

### Create Dataframe for Pass Plays Ending in a Sack

In [96]:
# create column list for sacks dataframe to help us compute num of sacks and sack yards lost 
sack_columns = ['passer_player_id'
               ,'passer_player_name'
               ,'posteam'
               ,'year'
               ,'week'
               ,'sack'
               ,'yards_gained'
               ]

# filter for pass plays that ended in a sack and select relevant columns
sacks = qb_play_by_play_df.loc[qb_play_by_play_df['sack'] == 1, sack_columns]

# remove sack and yards_gained from sack_columns
sck_cols_reduce = [i for i in sack_columns if i not in ('sack','yards_gained')]

# calculate num of sacks and sack yards lost by year, week, player
sacks = sacks.groupby(sck_cols_reduce, as_index=False)[['sack','yards_gained']].sum()

# rename yards_gained and sack columns
sacks.rename(columns = {'yards_gained':'sack_yrd_lost', 'sack':'sacks'}, inplace = True)

### Create Dataframe for Pass Plays Ending in a Red Zone TD

In [97]:
# filter for pass plays that ended in a red zone TD
red_zone_td = qb_play_by_play_df[(qb_play_by_play_df['yardline_100'] <= 20) & (qb_play_by_play_df['touchdown'] == 1)]

# calculate num of red zone TDs by year, week, player
red_zone_td = red_zone_td.groupby(sck_cols_reduce, as_index=False)[['touchdown']].sum()

# rename touchdown column
red_zone_td.rename(columns = {'touchdown':'rdz_td'}, inplace = True)

### Merge Passing Plays, Sacks, and Red Zone Dataframes

In [98]:
# group by player, team, week, year and sum the results for a weekly stat line for each player
qb_play_by_play_df = qb_play_by_play_df.groupby(sck_cols_reduce, as_index=False).sum()

# merge qb_play_by_play_df and sacks dataframes
qb_play_by_play_df = qb_play_by_play_df.merge(sacks, on = sck_cols_reduce, how = 'left')

# merge qb_play_by_play_df and red_zone_td dataframes
qb_play_by_play_df = qb_play_by_play_df.merge(red_zone_td, on = sck_cols_reduce, how = 'left')

# fill nulls with 0 since some players aren't sacked or have red zone TDs every game
qb_play_by_play_df[['sacks','sack_yrd_lost','rdz_td']] = qb_play_by_play_df[['sacks','sack_yrd_lost','rdz_td']].fillna(0)

### Create Dataframe of Team Yards after Catch and Merge wth Passing Plays Dataframe

In [99]:
# calculate team yards after catch by year, week
team_yac = qb_play_by_play_df.groupby(['posteam', 'year', 'week'], as_index=False)[['yards_after_catch']].sum()

# merge qb_play_by_play_df and team_yac dataframes
qb_play_by_play_df = qb_play_by_play_df.merge(team_yac
                                              ,on = ['posteam', 'year', 'week']
                                              ,how = 'left'
                                              ,suffixes = ('_ind', '_team')
                                             )

### Calculate Advanced Passing Metrics

In [100]:
# calculate completed air yards
qb_play_by_play_df['comp_air_yard'] = qb_play_by_play_df['yards_gained'] - qb_play_by_play_df['yards_after_catch_team']

# calculate completion percentage
qb_play_by_play_df['compl_rate'] = qb_play_by_play_df['complete_pass'] / (qb_play_by_play_df['incomplete_pass'] +\
                                                                          qb_play_by_play_df['complete_pass'])

# calculate adjusted net yards per attempt
qb_play_by_play_df['adj_net_yrd_per_att'] = (qb_play_by_play_df['yards_gained']+(20*qb_play_by_play_df['touchdown'])-\
                                            (45*qb_play_by_play_df['interception'])-qb_play_by_play_df['sack_yrd_lost'])/\
                                            ((qb_play_by_play_df['incomplete_pass']+qb_play_by_play_df['complete_pass'])+\
                                            qb_play_by_play_df['sacks'])

# calculate TD to passing attempts ratio
qb_play_by_play_df['td_to_att_ratio'] = qb_play_by_play_df['touchdown'] / (qb_play_by_play_df['incomplete_pass'] +\
                                                                           qb_play_by_play_df['complete_pass'])

# calculate INT to TD ratio 
qb_play_by_play_df['int_to_td_ratio'] = qb_play_by_play_df['interception'] / qb_play_by_play_df['touchdown']

# fill nulls with 0 since players can throw 0 TDs and 0 INTs during a game
qb_play_by_play_df['int_to_td_ratio'] = qb_play_by_play_df['int_to_td_ratio'].fillna(0)

# calculate total turnovers
qb_play_by_play_df['total_to'] = qb_play_by_play_df['interception'] + qb_play_by_play_df['fumble_lost']

# calculate passing yards per attempt
qb_play_by_play_df['pass_yrd_per_att'] = qb_play_by_play_df['yards_gained'] / (qb_play_by_play_df['incomplete_pass'] +\
                                                                               qb_play_by_play_df['complete_pass'])

# calculate air yards per attempt
qb_play_by_play_df['air_yrd_per_att'] = qb_play_by_play_df['air_yards'] / (qb_play_by_play_df['incomplete_pass'] +\
                                                                           qb_play_by_play_df['complete_pass'])

# remove any text after the players' last names such as Jr or II.  needed to join logic
qb_play_by_play_df['passer_player_name'] = qb_play_by_play_df['passer_player_name'].str.extract(r'(.*?)(?=$|\s)')

qb_play_by_play_df.head(3)

,passer_player_id,passer_player_name,posteam,year,week,incomplete_pass,complete_pass,pass_attempt,passing_yards,air_yards,yards_after_catch_ind,yards_gained,touchdown,interception,fumble_lost,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,qb_hit,sack,ep,epa,air_epa,comp_air_epa,qb_epa,air_wpa,comp_air_wpa,yardline_100,sacks,sack_yrd_lost,rdz_td,yards_after_catch_team,comp_air_yard,compl_rate,adj_net_yrd_per_att,td_to_att_ratio,int_to_td_ratio,total_to,pass_yrd_per_att,air_yrd_per_att
0,00-0019596,T.Brady,NE,2018,1,12.0,26.0,41.0,277.0,320.0,93.0,267.0,3.0,1.0,1.0,26,2,41.0,0,0,0,5.0,2.0,91.356903,2.922566,19.214496,9.002655,8.182894,0.017964,0.015368,2071.0,2.0,-10.0,2.0,93.0,174.0,0.684211,7.300000,0.078947,0.333333,2.0,7.026316,8.421053
1,00-0019596,T.Brady,NE,2018,2,11.0,24.0,37.0,234.0,213.0,136.0,220.0,2.0,0.0,1.0,26,1,37.0,0,0,0,4.0,2.0,73.112285,4.945607,10.703469,-1.011584,4.945607,0.021994,0.023892,1905.0,2.0,-14.0,1.0,136.0,84.0,0.685714,7.405405,0.057143,0.000000,1.0,6.285714,6.085714
2,00-0019596,T.Brady,NE,2018,3,11.0,14.0,28.0,133.0,333.0,48.0,120.0,1.0,1.0,0.0,21,4,28.0,0,0,0,6.0,2.0,36.009813,-10.869923,24.549755,2.058606,-10.869923,0.085198,0.082922,1723.0,2.0,-13.0,1.0,48.0,72.0,0.560000,4.000000,0.040000,1.000000,1.0,4.800000,13.320000


### Create Dataframe of QB Fantasy Football Statistics

In [101]:
# create column list of specific fantasy football statistics
qb_ff_stats = ['pass_comp_ff'
              ,'pass_incomp_ff'
              ,'pass_5_yrd_ff'
              ,'pass_yrd_300_399_ff'
              ,'pass_yrd_400+_ff'
              ,'pass_td_ff'
              ,'pass_50_yrd_td_ff'
              ,'pass_2pt_con_ff'
              ,'pass_int_ff'
              ,'rush_5_yrd_ff'
              ,'rush_yrd_100_199_ff'
              ,'rush_yrd_200+_ff'
              ,'rush_td_ff'
              ,'rush_50_yrd_td_ff'
              ,'rush_2pt_con_ff'
              ,'fum_lost_ff'
              ,'actual_points'
              ]

# create column list of specific football statistics
qb_stats = ['pass_comp'
           ,'pass_incomp'
           ,'pass_yrd'
           ,'pass_td'
           ,'pass_2pt_con'
           ,'pass_int'
           ,'rush_att'
           ,'rush_yrd'
           ,'rush_td'
           ,'rush_2pt_con'
           ,'fum_lost'
           ]

# filter by position_name is equal to QB + the column lists created above
qb_df = df.loc[df['position_name'] == 'QB', player_columns + qb_stats + qb_ff_stats]
qb_df.head()

,year,week,player,short_name,position_name,pro_team,pro_team_abv,pass_comp,pass_incomp,pass_yrd,pass_td,pass_2pt_con,pass_int,rush_att,rush_yrd,rush_td,rush_2pt_con,fum_lost,pass_comp_ff,pass_incomp_ff,pass_5_yrd_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_td_ff,pass_50_yrd_td_ff,pass_2pt_con_ff,pass_int_ff,rush_5_yrd_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rush_td_ff,rush_50_yrd_td_ff,rush_2pt_con_ff,fum_lost_ff,actual_points
5,2018,1,Matthew Stafford,M.Stafford,QB,Detroit Lions,DET,27.0,19.0,286.0,1.0,0.0,4.0,1.0,6.0,0.0,0.0,0.0,10.8,-3.8,5.7,0.0,0.0,6.0,0.0,0.0,-8.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,11.3
12,2018,1,Ben Roethlisberger,B.Roethlisberger,QB,Pittsburgh Steelers,PIT,23.0,18.0,335.0,1.0,0.0,3.0,3.0,16.0,0.0,0.0,2.0,9.2,-3.6,6.7,3.0,0.0,6.0,0.0,0.0,-6.0,1.8,0.0,0.0,0.0,0.0,0.0,-4.0,13.1
29,2018,1,Matt Ryan,M.Ryan,QB,Atlanta Falcons,ATL,21.0,22.0,251.0,0.0,0.0,1.0,2.0,8.0,0.0,0.0,0.0,8.4,-4.4,5.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,7.6
32,2018,1,Jimmy Garoppolo,J.Garoppolo,QB,San Francisco 49ers,SF,15.0,18.0,261.0,1.0,0.0,3.0,2.0,6.0,0.0,0.0,0.0,6.0,-3.6,5.2,0.0,0.0,6.0,0.0,0.0,-6.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,8.2
44,2018,1,Aaron Rodgers,A.Rodgers,QB,Greenbay Packers,GB,20.0,10.0,286.0,3.0,0.0,0.0,1.0,15.0,0.0,0.0,0.0,8.0,-2.0,5.7,0.0,0.0,18.0,3.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,34.5


### Merge Rushing Plays, Passing Plays, and Fantasy Football Statistics Dataframes

In [102]:
# merge qb_df and qb_play_by_play_df
qb_df = qb_df.merge(qb_play_by_play_df, left_on = ['year', 'week', 'short_name', 'pro_team_abv'], 
                                        right_on = ['year', 'week', 'passer_player_name', 'posteam'], how = 'left')

# merge qb_df and rb_df to grabs QBs' rushing stats
qb_df = qb_df.merge(rb_df, left_on = ['year', 'week', 'short_name', 'pro_team_abv'], 
                           right_on = ['year', 'week', 'rusher_player_name', 'posteam'], how = 'left')

# create updated column list of specific football statistics
upd_qb_stats = ['pass_comp'
               ,'pass_incomp'
               ,'pass_yrd'
               ,'air_yards'
               ,'pass_td'
               ,'rdz_td'
               ,'pass_2pt_con'
               ,'pass_int'
               ,'rush_att'
               ,'rush_yrd'
               ,'rush_td'
               ,'rush_2pt_con'
               ,'fum_lost'
               ]

# create column list of additional passing statistics
add_qb_stats = ['shotgun'
               ,'no_huddle'
               ,'qb_dropback'
               ,'qb_kneel'
               ,'qb_spike'
               ,'qb_scramble'
               ,'qb_hit'
               ,'sack'
               ,'ep'
               ,'epa'
               ,'air_epa'
               ,'comp_air_epa'
               ,'qb_epa'
               ,'air_wpa'
               ,'comp_air_wpa'
               ]

# create column list of advanced metrics for QBs
qb_advanced_metrics = ['compl_rate'
                      ,'pass_yrd_per_att'
                      ,'air_yrd_per_att'
                      ,'adj_net_yrd_per_att' 
                      ,'comp_air_yard'                       
                      ,'td_to_att_ratio'
                      ,'int_to_td_ratio'
                      ,'total_to'
                      ,'yrd_per_rush'
                      ,'rush_share'
                      ]

# select relevant columns
qb_df = qb_df[player_columns + upd_qb_stats + qb_advanced_metrics + add_qb_stats + qb_ff_stats]

# create list of columns to fill nulls with 0 since some players didn't throw or complete any passes during a game
null_to_zero = ['compl_rate'
               ,'rdz_td'
               ,'pass_yrd_per_att'
               ,'air_yrd_per_att'
               ,'adj_net_yrd_per_att'
               ,'comp_air_yard'
               ,'air_yards'
               ,'td_to_att_ratio'
               ,'int_to_td_ratio'
               ,'total_to'
               ,'yrd_per_rush'
               ,'rush_share'
               ,'shotgun'
               ,'no_huddle'
               ,'qb_dropback'
               ,'qb_kneel'
               ,'qb_spike'
               ,'qb_scramble'
               ,'qb_hit'
               ,'sack'
               ,'ep'
               ,'epa'
               ,'air_epa'
               ,'comp_air_epa'
               ,'qb_epa'
               ,'air_wpa'
               ,'comp_air_wpa'
               ]

# fill nulls with 0
qb_df[null_to_zero] = qb_df[null_to_zero].fillna(0)

# replace infinite values with 0
qb_df = qb_df.replace([-np.inf, np.inf], 0)

# update column type to category
qb_df['year'] = qb_df['year'].astype('category')
qb_df['week'] = qb_df['week'].astype('category')

qb_df.head()

,year,week,player,short_name,position_name,pro_team,pro_team_abv,pass_comp,pass_incomp,pass_yrd,air_yards,pass_td,rdz_td,pass_2pt_con,pass_int,rush_att,rush_yrd,rush_td,rush_2pt_con,fum_lost,compl_rate,pass_yrd_per_att,air_yrd_per_att,adj_net_yrd_per_att,comp_air_yard,td_to_att_ratio,int_to_td_ratio,total_to,yrd_per_rush,rush_share,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,qb_hit,sack,ep,epa,air_epa,comp_air_epa,qb_epa,air_wpa,comp_air_wpa,pass_comp_ff,pass_incomp_ff,pass_5_yrd_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_td_ff,pass_50_yrd_td_ff,pass_2pt_con_ff,pass_int_ff,rush_5_yrd_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rush_td_ff,rush_50_yrd_td_ff,rush_2pt_con_ff,fum_lost_ff,actual_points
0,2018,1,Matthew Stafford,M.Stafford,QB,Detroit Lions,DET,27.0,19.0,286.0,354.0,1.0,0.0,0.0,4.0,1.0,6.0,0.0,0.0,0.0,0.642857,6.809524,8.428571,3.476190,142.0,0.047619,2.0,5.0,6.000000,0.066667,42.0,27.0,46.0,0.0,0.0,0.0,4.0,0.0,114.416126,-15.735498,26.638698,-0.097134,-15.735498,0.008817,0.008817,10.8,-3.8,5.7,0.0,0.0,6.0,0.0,0.0,-8.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,11.3
1,2018,1,Ben Roethlisberger,B.Roethlisberger,QB,Pittsburgh Steelers,PIT,23.0,18.0,335.0,382.0,1.0,0.0,0.0,3.0,3.0,16.0,0.0,0.0,2.0,0.605263,8.236842,10.052632,5.238095,100.0,0.026316,3.0,5.0,5.333333,0.085714,42.0,4.0,45.0,0.0,0.0,0.0,5.0,4.0,65.470792,-6.626383,17.275068,3.820365,-6.626383,1.816574,0.917498,9.2,-3.6,6.7,3.0,0.0,6.0,0.0,0.0,-6.0,1.8,0.0,0.0,0.0,0.0,0.0,-4.0,13.1
2,2018,1,Matt Ryan,M.Ryan,QB,Atlanta Falcons,ATL,21.0,22.0,251.0,380.0,0.0,0.0,0.0,1.0,2.0,8.0,0.0,0.0,0.0,0.500000,5.357143,9.047619,4.478261,133.0,0.000000,0.0,1.0,4.000000,0.111111,34.0,5.0,47.0,0.0,0.0,0.0,13.0,4.0,96.311492,-12.027826,35.290724,7.462745,-12.027826,-0.189742,0.200121,8.4,-4.4,5.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,7.6
3,2018,1,Jimmy Garoppolo,J.Garoppolo,QB,San Francisco 49ers,SF,15.0,18.0,261.0,368.0,1.0,0.0,0.0,3.0,2.0,6.0,0.0,0.0,0.0,0.500000,7.900000,12.266667,5.030303,93.0,0.066667,1.5,3.0,3.000000,0.080000,26.0,4.0,36.0,0.0,0.0,0.0,8.0,3.0,71.477030,-13.548580,35.605712,7.101346,-13.548580,0.077075,0.087416,6.0,-3.6,5.2,0.0,0.0,6.0,0.0,0.0,-6.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,8.2
4,2018,1,Aaron Rodgers,A.Rodgers,QB,Greenbay Packers,GB,20.0,10.0,286.0,285.0,3.0,1.0,0.0,0.0,1.0,15.0,0.0,0.0,0.0,0.666667,8.900000,9.500000,10.812500,74.0,0.100000,0.0,0.0,15.000000,0.055556,31.0,13.0,32.0,0.0,0.0,0.0,4.0,2.0,64.437245,9.927506,10.545117,3.114478,9.927506,-0.806315,0.086284,8.0,-2.0,5.7,0.0,0.0,18.0,3.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,34.5


In [103]:
# view qb_df snapshot
qb_df.info()
qb_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1148 entries, 0 to 1147
Data columns (total 62 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   year                 1148 non-null   category
 1   week                 1148 non-null   category
 2   player               1148 non-null   object  
 3   short_name           1148 non-null   object  
 4   position_name        1148 non-null   object  
 5   pro_team             1148 non-null   object  
 6   pro_team_abv         1148 non-null   object  
 7   pass_comp            1148 non-null   float64 
 8   pass_incomp          1148 non-null   float64 
 9   pass_yrd             1148 non-null   float64 
 10  air_yards            1148 non-null   float64 
 11  pass_td              1148 non-null   float64 
 12  rdz_td               1148 non-null   float64 
 13  pass_2pt_con         1148 non-null   float64 
 14  pass_int             1148 non-null   float64 
 15  rush_att             

,pass_comp,pass_incomp,pass_yrd,air_yards,pass_td,rdz_td,pass_2pt_con,pass_int,rush_att,rush_yrd,rush_td,rush_2pt_con,fum_lost,compl_rate,pass_yrd_per_att,air_yrd_per_att,adj_net_yrd_per_att,comp_air_yard,td_to_att_ratio,int_to_td_ratio,total_to,yrd_per_rush,rush_share,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,qb_hit,sack,ep,epa,air_epa,comp_air_epa,qb_epa,air_wpa,comp_air_wpa,pass_comp_ff,pass_incomp_ff,pass_5_yrd_ff,pass_yrd_300_399_ff,pass_yrd_400+_ff,pass_td_ff,pass_50_yrd_td_ff,pass_2pt_con_ff,pass_int_ff,rush_5_yrd_ff,rush_yrd_100_199_ff,rush_yrd_200+_ff,rush_td_ff,rush_50_yrd_td_ff,rush_2pt_con_ff,fum_lost_ff,actual_points
count,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.0,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1148.0,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000
mean,19.667247,10.463415,224.633275,246.349303,1.526132,1.080139,0.078397,0.617596,3.219512,14.604530,0.175087,0.008711,0.182927,0.582810,6.341028,7.296854,6.458939,107.467770,0.049019,0.321596,0.880662,3.158016,0.118850,25.791812,3.351916,32.055749,0.0,0.124564,0.001742,4.496516,1.906794,65.200883,2.819345,14.812563,4.232276,3.427444,0.099821,0.107963,7.866899,-2.092683,4.458362,0.666376,0.209059,9.156794,0.290070,0.156794,-1.235192,1.608711,0.020906,0.0,1.050523,0.002613,0.017422,-0.365854,21.859059
std,9.728883,5.941994,115.984346,135.081876,1.293878,1.040013,0.284662,0.870481,3.211613,20.860864,0.443698,0.092965,0.433539,0.240502,3.193611,3.527090,3.671227,71.524511,0.043132,0.593981,1.051694,3.653697,0.104548,13.478170,4.622811,15.295133,0.0,0.401813,0.041721,3.200777,1.748091,34.179800,10.126353,11.400692,6.283935,10.002904,0.254762,0.199179,3.891553,1.188399,2.310933,1.247569,1.001231,7.763268,0.938569,0.569324,1.740962,2.441703,0.249670,0.0,2.662188,0.088542,0.185930,0.867078,13.868927
min,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-3.241379,-141.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-1.387298,-31.661474,-10.756646,-17.295740,-31.661474,-1.606985,-1.194717,0.000000,-5.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-4.000000,-8.100000
25%,16.000000,7.000000,170.000000,179.750000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.567568,5.236604,6.000000,4.561875,62.750000,0.000000,0.000000,0.000000,0.000000,0.037037,19.000000,0.000000,27.000000,0.0,0.000000,0.000000,2.000000,0.000000,48.816398,-2.471217,5.651746,0.000000,-1.757660,0.000000,0.000000,6.400000,-2.800000,3.400000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,12.000000
50%,21.000000,11.000000,241.000000,255.000000,1.000000,1.000000,0.000000,0.000000,3.000000,6.500000,0.000000,0.000000,0.000000,0.650000,6.775510,7.857744,6.866667,110.000000,0.045455,0.000000,1.000000,2.500000,0.103448,27.000000,2.000000,35.000000,0.0,0.000000,0.000000,4.000000,2.000000,68.785759,1.796051,14.680065,3.534795,2.656917,0.062967,0.069328,8.400000,-2.200000,4.800000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.600000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,21.850000
75%,26.000000,14.000000,303.000000,337.000000,2.000000,2.000000,0.000000,1.000000,5.000000,21.000000,0.000000,0.000000,0.000000,0.722222,8.200000,9.503378,8.842

In [104]:
# double check nulls
list(qb_df.isnull().sum(axis = 0))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### Save Dataframe

In [105]:
# save to csv
qb_df.to_csv("qb_feature_matrix.csv", index = False)

# Create Dataframe for Defenses

##### Advance Defense Metrics: 
* **Rush Yards per Attempt Against** = total rushing yards against / total rushing attempts against
* **Passing Yards per Attempt Against** = total passing yards against / total passing attempts against
* **Total Yards per Play Against** = (total rushing yards against +  total passing yards against) / (total rushing attempts against +  total passing attempts against)
* **Total Plays Against** = total rushing attempts against +  total passing attempts against
* **Completion Percentage Against** = num of completed passes / num of passing attempts
* **Average Depth of Target (adot) Against** = total air yards against / num of team targets against
* **Receiver Air Conversion Ratio (racr) Against** = total passing yards against / total air yards against
* **Completed Air Yards (compl_rec_air_yrds) Against** = total passing yards against - yards after catch against
* **Total TD Rate Against** = (total passing TDs + total rushing TDs) / (total passing attempts + total rushing attempts)
* **Net Yards per Passing Attempt Against** = (passing yards against - sack yards) / (passing attempts against + num of sacks)
* **Adjusted Net Yards per Passing Attempt Against** = total passing yards against + (20 * num of TDs against) - (45 * num of INTs) - total sack yards / (num of passing attempts against + num of sacks)
* **TD to Passing Attempts Ratio Against** = num of passing TDs against / num of passing attempts against
* **INT to TD Ratio Against** = num of INTs / num of passing TDs against
* **Total Turnovers** = fumbles + INTs
* **% of Drives Ending in an Offensive Score Against** = num of drives ending in offensive score / total num of drives against
* **% of Drives Ending in a Turnover** = num of drives ending in turnover / total num of drives against

### Create Dataframe of Rushing Plays (against Defenses) and Compute Advanced Rushing Metrics

In [106]:
# filter for rush plays only and select relevant columns
def_rush_p_by_p_df = play_by_play_df.loc[play_by_play_df['rush_attempt'] == 1, ['defteam'
                                                                               ,'year'
                                                                               ,'week'
                                                                               ,'rush_attempt'
                                                                               ,'rushing_yards'
                                                                               ,'yards_gained'
                                                                               ,'touchdown'
                                                                               ]
                                        ]

# remove rows with no defteam
def_rush_p_by_p_df = def_rush_p_by_p_df.loc[def_rush_p_by_p_df['defteam'].notnull()]

# group by player, team, week, year and sum the results for a weekly stat line for each player
def_rush_p_by_p_df = def_rush_p_by_p_df.groupby(['defteam', 'year', 'week'], as_index=False).sum()

# rename columns
def_rush_p_by_p_df.rename(columns = {'rush_attempt' :'rush_att_agst'
                                    ,'rushing_yards':'rush_yrd_agst'
                                    ,'yards_gained' :'rush_yrd_gained_agst'
                                    ,'touchdown'    :'rush_td_agst'
                                    }, inplace = True
                         )

# calculate rush yards per attempt againt
def_rush_p_by_p_df['rsh_yrd_per_att_agst'] = def_rush_p_by_p_df['rush_yrd_agst'] / def_rush_p_by_p_df['rush_att_agst']

def_rush_p_by_p_df.head(3)

,defteam,year,week,rush_att_agst,rush_yrd_agst,rush_yrd_gained_agst,rush_td_agst,rsh_yrd_per_att_agst
0,ARI,2018,1,42.0,182.0,182.0,1.0,4.333333
1,ARI,2018,2,35.0,90.0,94.0,3.0,2.571429
2,ARI,2018,3,31.0,122.0,122.0,1.0,3.935484


### Create Dataframe of Passing Plays (against Defenses)

In [107]:
# filter for passing plays only and select relevant columns
def_pass_p_by_p_df = play_by_play_df.loc[play_by_play_df['pass_attempt'] == 1, ['defteam'
                                                                               ,'year'
                                                                               ,'week'
                                                                               ,'incomplete_pass'
                                                                               ,'complete_pass'
                                                                               ,'pass_attempt'
                                                                               ,'passing_yards'
                                                                               ,'air_yards'
                                                                               ,'yards_after_catch'
                                                                               ,'pass_length'
                                                                               ,'yards_gained'
                                                                               ,'pass_location'
                                                                               ,'touchdown'
                                                                               ,'interception'
                                                                               ,'fumble_lost'
                                                                               ,'shotgun'
                                                                               ,'no_huddle'
                                                                               ,'qb_dropback'
                                                                               ,'qb_kneel'
                                                                               ,'qb_spike'
                                                                               ,'qb_scramble'
                                                                               ,'qb_hit'
                                                                               ,'sack'
                                                                               ,'ep'
                                                                               ,'epa'
                                                                               ,'air_epa'
                                                                               ,'comp_air_epa'
                                                                               ,'qb_epa'
                                                                               ,'air_wpa'
                                                                               ,'comp_air_wpa'
                                                                               ,'yardline_100'
                                                                               ,'drive_ended_with_score'
                                                                               ,'drive_end_transition'
                                                                               ]
                                        ]

# remove rows with no defteam
def_pass_p_by_p_df = def_pass_p_by_p_df.loc[def_pass_p_by_p_df['defteam'].notnull()]

### Create Dataframe for Pass Plays Ending in a Sack

In [108]:
# filter for pass plays that ended in a sack and select relevant columns
def_sacks = def_pass_p_by_p_df.loc[def_pass_p_by_p_df['sack'] == 1, ['defteam','year','week','sack','yards_gained']]

# calculate num of sacks and sack yards by year, week, team
def_sacks = def_sacks.groupby(['defteam', 'year', 'week'], as_index = False).sum()

# rename yards_gained and sack columns
def_sacks.rename(columns = {'yards_gained':'sack_yrd', 'sack': 'sacks'}, inplace = True)

### Create Dataframe for Pass Plays Ending in a Red Zone TD

In [109]:
# filter for pass plays that ended in a red zone TD
def_red_zone_td = def_pass_p_by_p_df[(def_pass_p_by_p_df['yardline_100'] <= 20) & (def_pass_p_by_p_df['touchdown'] == 1)]

# calculate num of red zone TDs against by year, week, team
def_red_zone_td = def_red_zone_td.groupby(['defteam', 'year', 'week'], as_index = False)[['touchdown']].sum()

# rename touchdown column
def_red_zone_td.rename(columns = {'touchdown':'rdz_td_agst'}, inplace = True)

### Merge Passing Plays, Sacks, and Red Zone Dataframes

In [110]:
# group by team, week, year and sum the results for a weekly stat line for each defense
def_pass_p_by_p_df = def_pass_p_by_p_df.groupby(['defteam', 'year', 'week'], as_index = False).sum()

# merge def_pass_p_by_p_df and def_sacks dataframes
def_pass_p_by_p_df = def_pass_p_by_p_df.merge(def_sacks, on = ['defteam', 'year', 'week'], how = 'left')

# merge def_pass_p_by_p_df and def_red_zone_td dataframes
def_pass_p_by_p_df = def_pass_p_by_p_df.merge(def_red_zone_td, on = ['defteam', 'year', 'week'], how = 'left')

# create list of columns to fill nulls with 0 since some team didn't have a sack or have a red zone TD thrown against them
null_to_zero = ['sacks','sack_yrd','rdz_td_agst']

# fill nulls with 0
def_pass_p_by_p_df[null_to_zero] = def_pass_p_by_p_df[null_to_zero].fillna(0)

# rename columns
def_pass_p_by_p_df.rename(columns = {'incomplete_pass'   :'inc_pass_agst'
                                    ,'complete_pass'     :'comp_pass_agst'
                                    ,'pass_attempt'      :'pass_att_agst'
                                    ,'passing_yards'     :'pass_yrd_agst'
                                    ,'air_yards'         :'air_yrd_agst'
                                    ,'yards_after_catch' :'yac_agst'
                                    ,'pass_length'       :'pass_len_agst'
                                    ,'yards_gained'      :'pass_yrd_gained_agst'
                                    ,'pass_location'     :'pass_loc_agst'
                                    ,'touchdown'         :'pass_td_agst'
                                    ,'interception'      :'int'
                                    ,'fumble_lost'       :'fumbles'
                                    ,'shotgun'           :'shotgun_agst'
                                    ,'no_huddle'         :'no_huddle_agst'
                                    ,'qb_dropback'       :'qb_drop_agst'
                                    ,'qb_kneel'          :'qb_kneel_agst'
                                    ,'qb_spike'          :'qb_spike_agst'
                                    ,'qb_scramble'       :'qb_scramble_agst'
                                    ,'qb_hit'            :'qb_hit_agst'
                                    ,'ep'                :'ep_agst'
                                    ,'epa'               :'epa_agst'
                                    ,'air_epa'           :'air_epa_agst'
                                    ,'comp_air_epa'      :'comp_air_epa_agst'
                                    ,'qb_epa'            :'qb_epa_agst'
                                    ,'air_wpa'           :'air_wpa_agst'
                                    ,'comp_air_wpa'      :'comp_air_wpa_agst'
                                    }, inplace = True
                         )

### Compute Advanced Metrics for Passing Plays (against Defenses)

In [111]:
# calculate passing yards per attempt against
def_pass_p_by_p_df['pass_yrd_per_att_agst'] = def_pass_p_by_p_df['pass_yrd_gained_agst'] /\
                                             (def_pass_p_by_p_df['inc_pass_agst'] + def_pass_p_by_p_df['comp_pass_agst'])

# calculate passing completion percentage against
def_pass_p_by_p_df['compl_perc_agst'] = def_pass_p_by_p_df['comp_pass_agst'] /\
                                       (def_pass_p_by_p_df['inc_pass_agst'] + def_pass_p_by_p_df['comp_pass_agst'])

# calculate adot against
def_pass_p_by_p_df['adot_agst'] = def_pass_p_by_p_df['air_yrd_agst'] /\
                                 (def_pass_p_by_p_df['inc_pass_agst'] + def_pass_p_by_p_df['comp_pass_agst'])

# calculate racr against
def_pass_p_by_p_df['racr_agst'] = def_pass_p_by_p_df['pass_yrd_gained_agst'] / def_pass_p_by_p_df['air_yrd_agst']

# calcualte completed air yards against 
def_pass_p_by_p_df['compl_air_yrds_agst'] = def_pass_p_by_p_df['pass_yrd_gained_agst'] - def_pass_p_by_p_df['yac_agst']

# calculate net yards per pass attempt against
def_pass_p_by_p_df['net_yrd_per_att_agst'] = (def_pass_p_by_p_df['pass_yrd_gained_agst'] - def_pass_p_by_p_df['sack_yrd'])\
                                             / ((def_pass_p_by_p_df['inc_pass_agst']\
                                             + def_pass_p_by_p_df['comp_pass_agst'])\
                                             + def_pass_p_by_p_df['sacks'])
    
# calculate adjusted net yards per pass attempt against
def_pass_p_by_p_df['adj_net_yrd_per_att_agst'] = (def_pass_p_by_p_df['pass_yrd_gained_agst']\
                                                 + (20 * def_pass_p_by_p_df['pass_td_agst'])\
                                                 - (45 * def_pass_p_by_p_df['int'])\
                                                 - def_pass_p_by_p_df['sack_yrd'])\
                                                 / ((def_pass_p_by_p_df['inc_pass_agst']\
                                                 + def_pass_p_by_p_df['comp_pass_agst'])\
                                                 + def_pass_p_by_p_df['sacks'])

# calculate passing TD to attempt ratio
def_pass_p_by_p_df['pass_td_to_att_ratio_agst'] = def_pass_p_by_p_df['pass_td_agst'] /(def_pass_p_by_p_df['inc_pass_agst']\
                                                                                     +def_pass_p_by_p_df['comp_pass_agst'])

# calculate INT to TD ratio against
def_pass_p_by_p_df['int_to_td_ratio_agst'] = def_pass_p_by_p_df['int'] / def_pass_p_by_p_df['pass_td_agst']

### Merge Rushing Plays and Passing Plays Dataframes and Compute More Advanced Metrics

In [112]:
# merge def_pass_p_by_p_df and def_rush_p_by_p_df dataframes
def_p_by_p_df = def_rush_p_by_p_df.merge(def_pass_p_by_p_df, on = ['defteam', 'year', 'week'], how = 'left')

# calcualte total turnovers
def_p_by_p_df['total_to'] = def_p_by_p_df['int'] + def_p_by_p_df['fumbles']

# calculate TD rate against 
def_p_by_p_df['td_rate_agst'] = (def_p_by_p_df['pass_td_agst'] + def_p_by_p_df['rush_td_agst'])  /\
                                (def_p_by_p_df['rush_att_agst'] + def_p_by_p_df['pass_att_agst'])
                                      
# calculate total yards per play against 
def_p_by_p_df['yrd_per_play_agst'] = (def_p_by_p_df['rush_yrd_agst'] + def_p_by_p_df['pass_yrd_gained_agst']) /\
                                     (def_p_by_p_df['rush_att_agst'] + def_p_by_p_df['pass_att_agst'])

# calculate plays against
def_p_by_p_df['plays_agst'] = def_p_by_p_df['rush_att_agst'] + def_p_by_p_df['pass_att_agst']

###  Create Dataframe to Compute Number of Penalties and Penalty Yards

In [113]:
# filter for plays where the defensive team is equal to the penalty team
# if there is no penalty on the penalty_team column is null
def_pen = play_by_play_df.loc[play_by_play_df['penalty_team'] == play_by_play_df['defteam'], ['defteam'
                                                                                              ,'penalty_team'
                                                                                              ,'year'
                                                                                              ,'week'
                                                                                              ,'penalty_yards'
                                                                                             ]
                             ]

# create column to count number of penalties
def_pen['num_penalty'] = 1

# calculate num of penalties and penalty yards by year, week, team
def_pen = def_pen.groupby(['defteam', 'penalty_team', 'year', 'week'], as_index=False).sum()

# merge def_p_by_p_df and def_pen dataframes
def_p_by_p_df = def_p_by_p_df.merge(def_pen, on=['defteam', 'year', 'week'], how='left')

### Create Dataframes to Compute Total Drives and Drives Ending in a Score or Turnover

In [114]:
# create list of columns to compute total number of drives, drives ending in a score, and drives ending in a turnover
drv_columns = ['posteam', 'defteam', 'year', 'week', 'drive']

# calculate total number of drive against
tot_drv_agst = play_by_play_df[drv_columns].groupby(drv_columns[:-1], as_index=False).agg({'drive': 'nunique'})

# rename drive column
tot_drv_agst.rename(columns = {'drive' :'tot_drives_agst'}, inplace = True)

# filter for drives that ended in a FG or a TD
drv_end_scr = play_by_play_df.loc[(play_by_play_df['drive_ended_with_score'] == 1)
                                &((play_by_play_df['rush_attempt'] == 1)
                                | (play_by_play_df['pass_attempt'] == 1)
                                |(play_by_play_df['play_type'] == 'field_goal'))
                                &((play_by_play_df['drive_end_transition'] == 'TOUCHDOWN') 
                                | (play_by_play_df['drive_end_transition'] == 'FIELD_GOAL')), drv_columns[1:]]

# calculate number drives that ended in a FG or a TD
drv_end_scr = drv_end_scr.groupby(['defteam', 'year', 'week'], as_index=False).agg({'drive': 'nunique'})

# rename drive column
drv_end_scr.rename(columns = {'drive' :'drv_end_scr'}, inplace = True)

# filter for drives that ended in an INT, funble, or a blocked kick
drv_end_to = play_by_play_df.loc[(play_by_play_df['drive_ended_with_score'] == 0)
                               &((play_by_play_df['rush_attempt'] == 1)
                               | (play_by_play_df['pass_attempt'] == 1)
                               | (play_by_play_df['play_type'] == 'punt')
                               | (play_by_play_df['play_type'] == 'extra_point')
                               | (play_by_play_df['play_type'] == 'field_goal'))
                               &((play_by_play_df['drive_end_transition'] == 'INTERCEPTION') 
                               | (play_by_play_df['drive_end_transition'] == 'FUMBLE')
                               | (play_by_play_df['drive_end_transition'] == 'BLOCKED_PUNT')
                               | (play_by_play_df['drive_end_transition'] == 'BLOCKED_FG')
                               | (play_by_play_df['drive_end_transition'] == 'BLOCKED_PUNT_DOWNS')
                               | (play_by_play_df['drive_end_transition'] == 'BLOCKED_FG_DOWNS')), drv_columns[1:]]

# calculate number drives that ended in an INT, funble, or a blocked kick
drv_end_to = drv_end_to.groupby(['defteam', 'year', 'week'], as_index=False).agg({'drive': 'nunique'})

# rename drive column
drv_end_to.rename(columns = {'drive' :'drv_end_to'}, inplace = True)

### Merge Drives and Play by Play Dataframes and Compute More Advanced Metrics

In [115]:
# merge def_p_by_p_df, tot_drv_agst, drv_end_scr, drv_end_to dataframes
def_p_by_p_df = def_p_by_p_df.merge(tot_drv_agst, on = ['defteam', 'year', 'week'], how = 'left')\
                             .merge(drv_end_scr,  on = ['defteam', 'year', 'week'], how = 'left')\
                             .merge(drv_end_to,   on = ['defteam', 'year', 'week'], how = 'left')

# calcule percentage of drives ending in TD or FG
def_p_by_p_df['drv_end_scr_perc'] = def_p_by_p_df['drv_end_scr'] / def_p_by_p_df['tot_drives_agst']

# calcule percentage of drives ending in INT, fumble, or blocked kick
def_p_by_p_df['drv_end_to_perc'] = def_p_by_p_df['drv_end_to'] / def_p_by_p_df['tot_drives_agst']

# create list of columns to fill nulls with 0 since defenses don't record every statistic every game
null_to_zero = ['int_to_td_ratio_agst'
                ,'penalty_team'
                ,'penalty_yards'
                ,'num_penalty'
                ,'drv_end_scr'
                ,'drv_end_to'
                ,'drv_end_scr_perc'
                ,'drv_end_to_perc'
               ]

# fill nulls with 0
def_p_by_p_df[null_to_zero] = def_p_by_p_df[null_to_zero].fillna(0)

def_p_by_p_df.head(3)

,defteam,year,week,rush_att_agst,rush_yrd_agst,rush_yrd_gained_agst,rush_td_agst,rsh_yrd_per_att_agst,inc_pass_agst,comp_pass_agst,pass_att_agst,pass_yrd_agst,air_yrd_agst,yac_agst,pass_yrd_gained_agst,pass_td_agst,int,fumbles,shotgun_agst,no_huddle_agst,qb_drop_agst,qb_kneel_agst,qb_spike_agst,qb_scramble_agst,qb_hit_agst,sack,ep_agst,epa_agst,air_epa_agst,comp_air_epa_agst,qb_epa_agst,air_wpa_agst,comp_air_wpa_agst,yardline_100,drive_ended_with_score,sacks,sack_yrd,rdz_td_agst,pass_yrd_per_att_agst,compl_perc_agst,adot_agst,racr_agst,compl_air_yrds_agst,net_yrd_per_att_agst,adj_net_yrd_per_att_agst,pass_td_to_att_ratio_agst,int_to_td_ratio_agst,total_to,td_rate_agst,yrd_per_play_agst,plays_agst,penalty_team,penalty_yards,num_penalty,posteam,tot_drives_agst,drv_end_scr,drv_end_to,drv_end_scr_perc,drv_end_to_perc
0,ARI,2018,1,42.0,182.0,182.0,1.0,4.333333,9.0,21.0,33.0,255.0,101.0,180.0,247.0,2.0,0.0,1.0,27,0,33.0,0,0,0,9.0,3.0,67.901473,9.827953,-9.453579,-6.206144,13.331767,0.000251,0.000251,1630.0,20.0,3.0,-8.0,2.0,8.233333,0.700000,3.366667,2.445545,67.0,7.727273,8.939394,0.066667,0.0,1.0,0.040000,5.72000,75.0,ARI,62.0,8.0,WAS,10,4.0,1.0,0.400000,0.100000
1,ARI,2018,2,35.0,90.0,94.0,3.0,2.571429,7.0,24.0,35.0,354.0,302.0,149.0,344.0,1.0,1.0,0.0,22,8,35.0,0,0,0,5.0,2.0,97.261271,16.001311,20.761553,11.291389,16.001311,-0.607202,0.106343,1382.0,17.0,2.0,-12.0,1.0,11.096774,0.774194,9.741935,1.139073,195.0,10.787879,10.030303,0.032258,1.0,1.0,0.057143,6.20000,70.0,ARI,18.0,4.0,LA,10,5.0,1.0,0.500000,0.100000
2,ARI,2018,3,31.0,122.0,122.0,1.0,3.935484,10.0,24.0,38.0,220.0,338.0,109.0,194.0,0.0,1.0,1.0,35,1,38.0,0,0,0,4.0,3.0,81.318723,-3.248134,12.633631,-9.592708,-3.248134,0.461819,0.458703,1841.0,21.0,3.0,-26.0,0.0,5.705882,0.705882,9.941176,0.573964,85.0,5.945946,4.729730,0.000000,inf,2.0,0.014493,4.57971,69.0,ARI,33.0,4.0,CHI,11,4.0,2.0,0.363636,0.181818


### Create Dataframe of Defensive Fantasy Football Statistics

In [116]:
# create list of specific fantasy football statistics
def_ff_stats = ['def_st_0_pts_alw_ff'
               ,'def_st_1_6_pts_alw_ff'
               ,'def_st_7_13_pts_alw_ff'
               ,'def_st_14_17_pts_alw_ff'
               ,'def_st_22_27_pts_alw_ff'
               ,'def_st_28_34_pts_alw_ff'
               ,'def_st_35_45_pts_alw_ff'
               ,'def_st_46+_pts_alw_ff'
               ,'def_st_0_99_yrd_alw_ff'
               ,'def_st_100_199_yrd_alw_ff'
               ,'def_st_200_299_yrd_alw_ff'
               ,'def_st_350_399_yrd_alw'
               ,'def_st_400_449_yrd_alw_ff'
               ,'def_st_450_499_yrd_alw_ff'
               ,'def_st_500_549_yrd_alw_ff'
               ,'def_st_550+_yrd_alw_ff'
               ,'def_st_int_ff'
               ,'def_st_fum_ff'
               ,'def_st_sack_ff'
               ,'def_st_blk_kick_ff'
               ,'def_st_safety_ff'
               ,'def_st_int_td_ff'
               ,'def_st_fum_ret_td_ff'
               ,'def_st_blk_td_ff'
               ,'def_st_kick_ret_td_ff'
               ,'def_st_punt_ret_td_ff'
               ,'actual_points' 
               ]

# create list of specific football statistics
def_stats = ['def_pts_alw'
            ,'def_tot_yrd_alw'
            ,'def_st_int'
            ,'def_st_fum'
            ,'def_st_sack'
            ,'def_st_safety'
            ,'def_st_blk_kick'
            ,'def_st_int_td'
            ,'def_st_fum_ret_td'
            ,'def_st_blk_td'
            ,'def_st_kick_ret_td'
            ,'def_st_punt_ret_td'
            ]

# filter by position_name is equal to DEF + the column lists created above
def_df = df.loc[df['position_name'] == 'DEF', player_columns + def_ff_stats + def_stats]

### Merge Defensive Play by Play and Fantasy Football Statistics Dataframes

In [117]:
# merge def_df and def_p_by_p_df dataframes
def_df = def_df.merge(def_p_by_p_df, left_on = ['year', 'week', 'pro_team_abv'], 
                                     right_on = ['year', 'week', 'defteam'], how = 'left')

# create list of football statistics against defenses
agst_stats = ['rush_att_agst'
             ,'rush_yrd_agst'
             ,'rush_yrd_gained_agst'
             ,'rush_td_agst'
             ,'inc_pass_agst'
             ,'comp_pass_agst'
             ,'pass_yrd_agst'
             ,'air_yrd_agst'
             ,'yac_agst'
             ,'pass_yrd_gained_agst'
             ,'pass_td_agst'
             ]

# create list of additional football statistics against defenses
add_def_stats = ['shotgun_agst'
                ,'no_huddle_agst' 
                ,'qb_drop_agst'
                ,'qb_kneel_agst'
                ,'qb_spike_agst'
                ,'qb_scramble_agst'
                ,'qb_hit_agst'
                ,'ep_agst'
                ,'epa_agst'
                ,'air_epa_agst'
                ,'comp_air_epa_agst'
                ,'qb_epa_agst'
                ,'air_wpa_agst'
                ,'comp_air_wpa_agst'
                ]

# create list of advanced defensive metrics
adv_def_stats = ['plays_agst'
                ,'rsh_yrd_per_att_agst'
                ,'compl_perc_agst'
                ,'pass_yrd_per_att_agst'
                ,'yrd_per_play_agst'
                ,'compl_air_yrds_agst'
                ,'racr_agst'
                ,'adot_agst'
                ,'net_yrd_per_att_agst'
                ,'adj_net_yrd_per_att_agst'
                ,'pass_td_to_att_ratio_agst'
                ,'rdz_td_agst'
                ,'td_rate_agst'
                ,'int_to_td_ratio_agst'
                ,'total_to'
                ,'sack_yrd'
                ,'num_penalty'
                ,'penalty_yards'
                ,'tot_drives_agst'
                ,'drv_end_scr'
                ,'drv_end_scr_perc'
                ,'drv_end_to'
                ,'drv_end_to_perc'
                ]

# select relevant columns
def_df = def_df[player_columns + def_stats + agst_stats + adv_def_stats + add_def_stats + def_ff_stats]

# fill nulls with 0
def_df = def_df.fillna(0)

# replace infinite values with 0
def_df = def_df.replace([-np.inf, np.inf], 0)

def_df.head(3)

,year,week,player,short_name,position_name,pro_team,pro_team_abv,def_pts_alw,def_tot_yrd_alw,def_st_int,def_st_fum,def_st_sack,def_st_safety,def_st_blk_kick,def_st_int_td,def_st_fum_ret_td,def_st_blk_td,def_st_kick_ret_td,def_st_punt_ret_td,rush_att_agst,rush_yrd_agst,rush_yrd_gained_agst,rush_td_agst,inc_pass_agst,comp_pass_agst,pass_yrd_agst,air_yrd_agst,yac_agst,pass_yrd_gained_agst,pass_td_agst,plays_agst,rsh_yrd_per_att_agst,compl_perc_agst,pass_yrd_per_att_agst,yrd_per_play_agst,compl_air_yrds_agst,racr_agst,adot_agst,net_yrd_per_att_agst,adj_net_yrd_per_att_agst,pass_td_to_att_ratio_agst,rdz_td_agst,td_rate_agst,int_to_td_ratio_agst,total_to,sack_yrd,num_penalty,penalty_yards,tot_drives_agst,drv_end_scr,drv_end_scr_perc,drv_end_to,drv_end_to_perc,shotgun_agst,no_huddle_agst,qb_drop_agst,qb_kneel_agst,qb_spike_agst,qb_scramble_agst,qb_hit_agst,ep_agst,epa_agst,air_epa_agst,comp_air_epa_agst,qb_epa_agst,air_wpa_agst,comp_air_wpa_agst,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_46+_pts_alw_ff,def_st_0_99_yrd_alw_ff,def_st_100_199_yrd_alw_ff,def_st_200_299_yrd_alw_ff,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,def_st_int_ff,def_st_fum_ff,def_st_sack_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_blk_td_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,actual_points
0,2018,1,Jaguars D/ST,J.D/ST,DEF,Jacksonville Jaguars,JAX,15.0,324.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,24.0,114.0,114.0,1.0,13.0,23.0,224.0,249.0,86.0,210.0,1.0,63.0,4.750000,0.638889,5.833333,5.142857,124.0,0.843373,6.916667,5.894737,5.236842,0.027778,0.0,0.031746,1.0,1.0,-14.0,5.0,73.0,12.0,4.0,0.333333,1.0,0.083333,33.0,5.0,39.0,0.0,0.0,0.0,6.0,49.355886,-11.798196,5.400742,-2.863654,-11.798196,0.087749,0.086177,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,21.0
1,2018,1,Ravens D/ST,R.D/ST,DEF,Baltimore Ravens,BAL,3.0,153.0,2.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,83.0,83.0,0.0,20.0,11.0,98.0,368.0,33.0,70.0,0.0,61.0,3.772727,0.354839,2.258065,2.508197,37.0,0.190217,11.870968,2.648649,0.216216,0.000000,0.0,0.000000,0.0,2.0,-28.0,4.0,38.0,15.0,1.0,0.066667,2.0,0.133333,39.0,11.0,39.0,0.0,0.0,0.0,9.0,31.496089,-27.864169,26.181205,1.746639,-27.864169,-0.004956,-0.004946,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0
2,2018,1,Vikings D/ST,V.D/ST,DEF,Minnesota Vikings,MIN,16.0,327.0,3.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,25.0,90.0,90.0,0.0,15.0,15.0,261.0,368.0,144.0,237.0,2.0,61.0,3.600000,0.500000,7.900000,5.360656,93.0,0.644022,12.266667,7.909091,5.030303,0.066667,0.0,0.032787,1.5,3.0,-24.0,4.0,37.0,12.0,4.0,0.333333,4.0,0.333333,26.0,4.0,36.0,0.0,0.0,0.0,8.0,71.477030,-13.548580,35.605712,7.101346,-13.548580,0.077075,0.087416,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,25.0


In [118]:
# view def_df snapshot
def_df.info()
def_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 841 entries, 0 to 840
Data columns (total 94 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       841 non-null    int64  
 1   week                       841 non-null    int64  
 2   player                     841 non-null    object 
 3   short_name                 841 non-null    object 
 4   position_name              841 non-null    object 
 5   pro_team                   841 non-null    object 
 6   pro_team_abv               841 non-null    object 
 7   def_pts_alw                841 non-null    float64
 8   def_tot_yrd_alw            841 non-null    float64
 9   def_st_int                 841 non-null    float64
 10  def_st_fum                 841 non-null    float64
 11  def_st_sack                841 non-null    float64
 12  def_st_safety              841 non-null    float64
 13  def_st_blk_kick            841 non-null    float64

,year,week,def_pts_alw,def_tot_yrd_alw,def_st_int,def_st_fum,def_st_sack,def_st_safety,def_st_blk_kick,def_st_int_td,def_st_fum_ret_td,def_st_blk_td,def_st_kick_ret_td,def_st_punt_ret_td,rush_att_agst,rush_yrd_agst,rush_yrd_gained_agst,rush_td_agst,inc_pass_agst,comp_pass_agst,pass_yrd_agst,air_yrd_agst,yac_agst,pass_yrd_gained_agst,pass_td_agst,plays_agst,rsh_yrd_per_att_agst,compl_perc_agst,pass_yrd_per_att_agst,yrd_per_play_agst,compl_air_yrds_agst,racr_agst,adot_agst,net_yrd_per_att_agst,adj_net_yrd_per_att_agst,pass_td_to_att_ratio_agst,rdz_td_agst,td_rate_agst,int_to_td_ratio_agst,total_to,sack_yrd,num_penalty,penalty_yards,tot_drives_agst,drv_end_scr,drv_end_scr_perc,drv_end_to,drv_end_to_perc,shotgun_agst,no_huddle_agst,qb_drop_agst,qb_kneel_agst,qb_spike_agst,qb_scramble_agst,qb_hit_agst,ep_agst,epa_agst,air_epa_agst,comp_air_epa_agst,qb_epa_agst,air_wpa_agst,comp_air_wpa_agst,def_st_0_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_22_27_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_46+_pts_alw_ff,def_st_0_99_yrd_alw_ff,def_st_100_199_yrd_alw_ff,def_st_200_299_yrd_alw_ff,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,def_st_int_ff,def_st_fum_ff,def_st_sack_ff,def_st_blk_kick_ff,def_st_safety_ff,def_st_int_td_ff,def_st_fum_ret_td_ff,def_st_blk_td_ff,def_st_kick_ret_td_ff,def_st_punt_ret_td_ff,actual_points
count,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.0,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000,841.000000
mean,2018.940547,9.674197,19.892985,317.429251,0.793103,0.555291,2.435196,0.039239,0.063020,0.078478,0.061831,0.007134,0.011891,0.010702,25.599287,109.546968,109.640904,0.848989,11.683710,21.819263,241.788347,272.928656,112.470868,224.600476,1.598098,62.688466,4.183763,0.648362,6.708148,5.300752,112.129608,0.874753,8.141878,6.741707,6.600373,0.049292,1.033294,0.038940,0.417905,1.167658,-17.350773,2.933413,27.656361,11.330559,3.809750,0.341226,1.454221,0.126019,29.910820,3.978597,36.946492,0.0,0.142687,0.003567,5.693222,70.724601,-0.383562,15.743551,3.578008,0.314137,0.123065,0.127070,0.118906,0.324614,0.492271,0.153389,-0.206897,-0.435196,-0.338882,-0.074911,0.008323,0.153389,0.279429,0.205707,-0.129608,-0.083829,-0.054697,-0.010702,2.379310,1.665874,2.435196,0.126040,0.078478,0.392390,0.309156,0.042806,0.071344,0.064209,9.700951
std,0.815787,4.756290,10.636548,112.382627,0.949937,0.744567,1.862767,0.194279,0.252747,0.277790,0.245882,0.084213,0.108458,0.113932,7.760244,51.591832,51.647726,0.908833,4.308058,6.251907,78.703832,97.651626,45.388161,81.285298,1.238354,9.703484,1.366147,0.104708,2.046791,1.254214,58.169875,0.352465,2.287319,1.895408,2.814706,0.040374,0.961811,0.023417,0.665714,1.112355,13.552786,1.865818,20.749632,1.919234,1.631374,0.151568,1.221088,0.101385,9.314883,4.820790,9.070530,0.0,0.423811,0.059655,2.927467,24.320280,11.266708,10.502802,6.591735,11.165344,0.281126,0.235600,1.084583,1.472922,1.111734,0.360576,0.405322,1.057129,1.257557,0.720681,0.241379,0.661179,0.448986,0.404458,0.336071,0.344757,0.3263

In [119]:
# double check nulls
list(def_df.isnull().sum(axis = 0))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### Save Dataframe

In [120]:
# save to csv
def_df.to_csv("def_feature_matrix.csv", index = False)

# Create Dataframe for Kickers

##### "Advanced" Kicking Metrics:
* **Total Team Offensive Plays** = team rushing attempts + team passing attempts
* **Total Team Offensive Yards** = team rushing yards + team passing yards
* **Total Team Offensive Scores** = team rushing TDs + team passing TDs + team FGs
* **Total Team Offensive Red Zone Scores** = team rushing TDs + team passing TDs + team FGs from 20 yard line to goal line

### Create Dataframe of Rushing and Passing Plays

In [121]:
# create list of relevant columns
kr_columns = ['posteam'
             ,'year'
             ,'week'
             ,'rush_attempt'
             ,'pass_attempt'
             ,'yards_gained'
             ]

# filter for rush plays only and select relevant columns
kr_rush_df = play_by_play_df.loc[play_by_play_df['rush_attempt'] == 1, kr_columns]

# remove rows with no posteam
kr_rush_df = kr_rush_df.loc[kr_rush_df['posteam'].notnull()]

# group by team, week, year and sum the results for a weekly stat line for each team
kr_rush_df = kr_rush_df.groupby(['posteam', 'year', 'week'], as_index=False)[['rush_attempt', 'yards_gained']].sum()

# rename yards_gained column
kr_rush_df = kr_rush_df.rename({'yards_gained': 'rushing_yards'}, axis=1)

# filter for pass plays only and select relevant columns
kr_pass_df = play_by_play_df.loc[play_by_play_df['pass_attempt'] == 1, kr_columns]

# remove rows with no posteam
kr_pass_df = kr_pass_df.loc[kr_pass_df['posteam'].notnull()]

# group by team, week, year and sum the results for a weekly stat line for each team
kr_pass_df = kr_pass_df.groupby(['posteam', 'year', 'week'], as_index=False)[['pass_attempt', 'yards_gained']].sum()

# rename yards_gained column
kr_pass_df = kr_pass_df.rename({'yards_gained': 'passing_yards'}, axis=1)

# create list of columns to group by
kr_columns_reduce = [i for i in kr_columns if i not in ('rush_attempt','pass_attempt','yards_gained')]

# merge kr_rush_df and kr_pass_df dataframes
kr_df = kr_rush_df.merge(kr_pass_df, on = kr_columns_reduce, how = 'left')

# calculate total offensive plays per week
kr_df['total_plays'] = kr_df['rush_attempt'] + kr_df['pass_attempt']

# calculate total offensive yards per week
kr_df['total_yards'] = kr_df['rushing_yards'] + kr_df['passing_yards']

kr_df.head(3)

,posteam,year,week,rush_attempt,rushing_yards,pass_attempt,passing_yards,total_plays,total_yards
0,ARI,2018,1,15.0,68.0,37.0,145.0,52.0,213.0
1,ARI,2018,2,15.0,54.0,28.0,83.0,43.0,137.0
2,ARI,2018,3,18.0,53.0,30.0,168.0,48.0,221.0


### Create Dataframe of Total Offensive Scores

In [122]:
# create list of relevant columns
scores_columns = ['posteam'
                 ,'year'
                 ,'week'
                 ,'touchdown'
                 ,'field_goal_result'
                 ]

# filter for plays that ended in a FG or a TD
total_scores = play_by_play_df.loc[(play_by_play_df['touchdown'] == 1)
                                  |(play_by_play_df['field_goal_result'] == 'made'), scores_columns]

# create new field_goal column so we can add it to the touchdown column
total_scores['field_goal'] = [1 if x == 'made' else 0 for x in total_scores['field_goal_result']]

# remove rows with no posteam
total_scores = total_scores.loc[total_scores['posteam'].notnull()]

# group by team, week, year and sum the results for a weekly stat line for each team
total_scores = total_scores.groupby(['posteam', 'year', 'week'], as_index=False).sum()

# calculate total offensive scores
total_scores['total_scores'] = total_scores['field_goal'] + total_scores['touchdown']

total_scores.head(3)

,posteam,year,week,touchdown,field_goal,total_scores
0,ARI,2018,1,1.0,0,1.0
1,ARI,2018,3,2.0,0,2.0
2,ARI,2018,4,2.0,1,3.0


### Create Dataframe for Total Offensive Score in the Red Zone

In [123]:
# create list of relevant columns
scores_rd_columns = ['posteam'
                    ,'year'
                    ,'week'
                    ,'touchdown'
                    ,'field_goal_result'
                    ]

# filter for drives that ended in a FG or a TD
total_scores_rd = play_by_play_df.loc[((play_by_play_df['touchdown'] == 1)
                                     | (play_by_play_df['field_goal_result'] == 'made'))
                                     & (play_by_play_df['yardline_100'] <= 20), scores_rd_columns]

# create new field_goal column so we can add it to the touchdown column
total_scores_rd['field_goal'] = [1 if x == 'made' else 0 for x in total_scores_rd['field_goal_result']]

# remove rows with no posteam
total_scores_rd = total_scores_rd.loc[total_scores_rd['posteam'].notnull()]

# group by team, week, year and sum the results for a weekly stat line for each team
total_scores_rd = total_scores_rd.groupby(['posteam', 'year', 'week'], as_index=False).sum()

# calculate total offensive scores
total_scores_rd['total_scores_rz'] = total_scores_rd['field_goal'] + total_scores_rd['touchdown']

total_scores_rd.head(3)

,posteam,year,week,touchdown,field_goal,total_scores_rz
0,ARI,2018,1,1.0,0,1.0
1,ARI,2018,4,1.0,1,2.0
2,ARI,2018,5,2.0,0,2.0


### Create Dataframe of Kicker Fantasy Football Statistics

In [124]:
# create list of fantasy football statistics
kick_ff_stats = ['pat_made_ff'
                ,'pat_miss_ff'
                ,'fg_made_0_39_ff'
                ,'fg_miss_0_39_ff'
                ,'fg_made_40_49_ff'
                ,'fg_miss_40_49_ff'
                ,'fg_made_50_59_ff'
                ,'actual_points'
                ]

# create list of actual football statistics
kick_stats = ['pat_con'
             ,'pat_att'
             ,'fg_con'
             ,'fg_att'
             ]

# filter by position_name is equal to KR + the column lists created above
kick_df = df.loc[df['position_name'] == 'KR', player_columns + kick_stats + kick_ff_stats]

### Merge Kicker Dataframes

In [125]:
# merge kick_df and kr_df dataframes
kick_df = kick_df.merge(kr_df, left_on = ['pro_team_abv', 'year', 'week'], 
                               right_on = ['posteam', 'year', 'week'], how = 'left')

# merge kick_df and total_scores dataframes
kick_df = kick_df.merge(total_scores, left_on = ['pro_team_abv', 'year', 'week'], 
                                      right_on = ['posteam', 'year', 'week'], how = 'left')

# merge kick_df and kr_df dataframes
kick_df = kick_df.merge(total_scores_rd, left_on = ['pro_team_abv', 'year', 'week'], 
                                         right_on = ['posteam', 'year', 'week'], how = 'left')

# fill nulls with 0 since some teams don't score every game
kick_df[['total_scores','total_scores_rz']] = kick_df[['total_scores','total_scores_rz']].fillna(0)

kick_df.head(3)

,year,week,player,short_name,position_name,pro_team,pro_team_abv,pat_con,pat_att,fg_con,fg_att,pat_made_ff,pat_miss_ff,fg_made_0_39_ff,fg_miss_0_39_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_50_59_ff,actual_points,posteam_x,rush_attempt,rushing_yards,pass_attempt,passing_yards,total_plays,total_yards,posteam_y,touchdown_x,field_goal_x,total_scores,posteam,touchdown_y,field_goal_y,total_scores_rz
0,2018,1,Chris Boswell,C.Boswell,KR,Pittsburgh Steelers,PIT,3.0,3.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,-1.0,0.0,2.0,PIT,35.0,159.0,45.0,313.0,80.0,472.0,PIT,3.0,0.0,3.0,PIT,1.0,0.0,1.0
1,2018,1,Harrison Butker,H.Butker,KR,Kansas City Chiefs,KC,5.0,5.0,1.0,1.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,9.0,KC,27.0,106.0,28.0,256.0,55.0,362.0,KC,4.0,1.0,5.0,KC,2.0,0.0,2.0
2,2018,1,Matt Prater,M.Prater,KR,Detroit Lions,DET,2.0,2.0,1.0,3.0,2.0,0.0,3.0,0.0,0.0,-1.0,0.0,4.0,DET,15.0,39.0,52.0,300.0,67.0,339.0,DET,3.0,1.0,4.0,DET,0.0,1.0,1.0


### Compute Advanced Kicking Metrics

In [126]:
# calculate FG percentage
kick_df['fg_perc'] = kick_df['fg_con'] / kick_df['fg_att']

# calculate PAT percentage
kick_df['pat_perc'] = kick_df['pat_con'] / kick_df['pat_att']

# calculate total kicking percentage
kick_df['kick_perc'] = (kick_df['pat_con'] + kick_df['fg_con']) / (kick_df['pat_att'] + kick_df['fg_att'])

# create list of advanced kicking metrics
adv_kick_stats = ['pat_perc', 'fg_perc', 'kick_perc', 'total_plays', 'total_yards', 'total_scores','total_scores_rz']

# reorder columns
kick_df = kick_df[player_columns + kick_stats + adv_kick_stats + kick_ff_stats]

# create list of columns to convert nulls to zero
null_to_zero = ['pat_perc', 'fg_perc', 'kick_perc', 'total_plays', 'total_yards']

# fill nulls with 0
kick_df[null_to_zero] = kick_df[null_to_zero].fillna(0)

# replace infinite values with 0
kick_df = kick_df.replace([-np.inf, np.inf], 0)

kick_df.head()

,year,week,player,short_name,position_name,pro_team,pro_team_abv,pat_con,pat_att,fg_con,fg_att,pat_perc,fg_perc,kick_perc,total_plays,total_yards,total_scores,total_scores_rz,pat_made_ff,pat_miss_ff,fg_made_0_39_ff,fg_miss_0_39_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_50_59_ff,actual_points
0,2018,1,Chris Boswell,C.Boswell,KR,Pittsburgh Steelers,PIT,3.0,3.0,0.0,1.0,1.0,0.000000,0.750,80.0,472.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,-1.0,0.0,2.0
1,2018,1,Harrison Butker,H.Butker,KR,Kansas City Chiefs,KC,5.0,5.0,1.0,1.0,1.0,1.000000,1.000,55.0,362.0,5.0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,9.0
2,2018,1,Matt Prater,M.Prater,KR,Detroit Lions,DET,2.0,2.0,1.0,3.0,1.0,0.333333,0.600,67.0,339.0,4.0,1.0,2.0,0.0,3.0,0.0,0.0,-1.0,0.0,4.0
3,2018,1,Robbie Gould,R.Gould,KR,San Francisco 49ers,SF,1.0,1.0,3.0,3.0,1.0,1.000000,1.000,61.0,327.0,5.0,2.0,1.0,0.0,6.0,0.0,4.0,0.0,0.0,11.0
4,2018,1,Greg Zuerlein,G.Zuerlein,KR,Los Angeles Rams,LA,3.0,3.0,4.0,5.0,1.0,0.800000,0.875,60.0,365.0,6.0,5.0,3.0,0.0,9.0,0.0,0.0,-1.0,0.0,16.0


In [127]:
# view kick_df snapshot
kick_df.info()
kick_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572 entries, 0 to 571
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              572 non-null    int64  
 1   week              572 non-null    int64  
 2   player            572 non-null    object 
 3   short_name        572 non-null    object 
 4   position_name     572 non-null    object 
 5   pro_team          572 non-null    object 
 6   pro_team_abv      572 non-null    object 
 7   pat_con           572 non-null    float64
 8   pat_att           572 non-null    float64
 9   fg_con            572 non-null    float64
 10  fg_att            572 non-null    float64
 11  pat_perc          572 non-null    float64
 12  fg_perc           572 non-null    float64
 13  kick_perc         572 non-null    float64
 14  total_plays       572 non-null    float64
 15  total_yards       572 non-null    float64
 16  total_scores      572 non-null    float64
 1

,year,week,pat_con,pat_att,fg_con,fg_att,pat_perc,fg_perc,kick_perc,total_plays,total_yards,total_scores,total_scores_rz,pat_made_ff,pat_miss_ff,fg_made_0_39_ff,fg_miss_0_39_ff,fg_made_40_49_ff,fg_miss_40_49_ff,fg_made_50_59_ff,actual_points
count,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000
mean,2018.945804,9.368881,2.547203,2.690559,1.568182,1.818182,0.841396,0.743969,0.866926,63.180070,362.265734,4.524476,3.075175,2.547203,-0.143357,2.910839,-0.076923,1.706294,-0.117133,0.297203,7.694930
std,0.819337,4.893823,1.556707,1.601351,1.160002,1.254827,0.330753,0.393280,0.248714,13.148515,105.113929,1.746980,1.501760,1.556707,0.393121,2.808685,0.419773,2.656867,0.337790,1.289507,4.381144
min,2018.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,-4.000000,0.000000,-2.000000,0.000000,-2.000000
25%,2018.000000,5.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.500000,0.800000,59.000000,305.000000,4.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,2019.000000,10.000000,3.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,65.000000,373.000000,5.000000,3.000000,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,8.000000
75%,2020.000000,14.000000,4.000000,4.000000,2.000000,3.000000,1.000000,1.000000,1.000000,70.000000,430.000000,6.000000,4.000000,4.000000,0.000000,6.000000,0.000000,4.000000,0.000000,0.000000,10.000000
max,2020.000000,17.000000,8.000000,8.000000,5.000000,5.000000,1.000000,1.000000,1.000000,95.000000,643.000000,9.000000,7.000000,8.000000,0.000000,12.000000,0.000000,12.000000,0.000000,10.000000,22.000000


In [128]:
# double check nulls
list(kick_df.isnull().sum(axis = 0))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### Save Dataframe

In [129]:
# save to csv
kick_df.to_csv("kr_feature_matrix.csv", index = False)

In [ ]:
import sys
!{sys.executable} -m pip install pip setuptools wheel

In [ ]:
import sys
!{sys.executable} -m pip install spacy

In [ ]:
import sys
!{sys.executable} -m pip install srsly==2.0.1

In [ ]:
import srsly

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
import sys
!{sys.executable} -m pip install ujson

In [ ]:
from zipfile import ZipFile

# z = ZipFile(yourfile)

# text_files = z.infolist()

# for text_file in text_files:
#     z.read(text_file.filename)

In [ ]:
test3 = pd.read_csv('../../Data_Science_Team/Projects/Pharmacy_FWA/orig_hist_df_w_labels.csv', nrows = 100)
output_dir = 'W:/GIG/- 2 Applied Analytics Group/Projects/Macquarie-Terr_Opt/pyaud/{}'.format(dataset)
test3.head()